In [18]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import random
import seaborn as sns
from tqdm import tqdm
from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
%matplotlib inline


import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Flatten)
from tensorflow.keras.layers import (Dense, Lambda, Dropout, Activation)
from keras.applications.resnet import preprocess_input

from tensorflow.keras.layers import (Conv2D, MaxPooling2D)
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer, RobustScaler

In [19]:
styles_df = pd.read_csv("styles.csv", on_bad_lines='skip')

STYLES_CSV_PATH = "styles.csv"
IMAGES_PATH = "images"

# print(styles_df.shape)
# print(styles_df.head())
# print(styles_df.isnull().sum())
# for column in styles_df.columns:
#     print(f"Unique values in '{column}':")
#     print(styles_df[column].value_counts())
#     print("\n")

In [20]:
styles_df = styles_df.dropna()

columns_to_drop = ['year', 'productDisplayName', 'masterCategory', 'articleType']
existing_columns = [col for col in columns_to_drop if col in styles_df.columns]

styles_df.drop(columns=existing_columns, inplace=True)

styles_df = styles_df[styles_df['id'].isin([int(i.split('.')[0]) for i in os.listdir(IMAGES_PATH)])]



In [21]:
print(styles_df.shape)
styles_df.head()

(44072, 6)


,id,gender,subCategory,baseColour,season,usage
0,15970,Men,Topwear,Navy Blue,Fall,Casual
1,39386,Men,Bottomwear,Blue,Summer,Casual
2,59263,Women,Watches,Silver,Winter,Casual
3,21379,Men,Bottomwear,Black,Fall,Casual
4,53759,Men,Topwear,Grey,Summer,Casual


In [22]:
# for column in styles_df.columns:
#     print(f"Unique values in '{column}':")
#     print(styles_df[column].value_counts())
#     print("\n")

In [23]:
# import matplotlib.pyplot as plt

# output_dir = "abc"  # Set your output directory here

# for column in styles_df.columns:
#     if column == 'id':
#         continue

#     # Create a new figure for each plot
#     plt.figure(figsize=(10, 6))  # Adjust size as needed
    
#     # Create the plot
#     ax = sns.countplot(data=styles_df, x=column, 
#                       order=styles_df[column].value_counts().index)
    
#     # Formatting
#     plt.title(f'Distribution of {column}', fontsize=14)
#     plt.xlabel(column, fontsize=12)
#     plt.ylabel('Count', fontsize=12)
#     plt.xticks(rotation=45, ha='right')  # Adjust rotation and alignment
    
#     # Adjust layout to prevent label cutoff
#     plt.tight_layout()
    
#     # Save in multiple formats (choose what you need)
#     filename = f"distribution_{column.lower().replace(' ', '_')}"
    
#     # Save as PDF (vector format)
#     plt.savefig(f"{output_dir}/{filename}.pdf", 
#                 bbox_inches='tight', 
#                 format='pdf')
    
#     # Save as PNG (high resolution)
#     plt.savefig(f"{output_dir}/{filename}.png", 
#                 dpi=300, 
#                 bbox_inches='tight',
#                 transparent=True)
    
#     # Close the figure to free memory
#     plt.close()

In [24]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# df=styles_df.copy()
# # Assuming you already have your DataFrame as 'df'
# attributes = ['gender', 'subCategory', 'baseColour', 'season', 'usage']

# for attr in attributes:
#     df[attr].value_counts().plot.pie(autopct='%1.1f%%', figsize=(8, 8))
#     plt.title(f"Distribution of {attr}")
#     plt.ylabel("")  
#     plt.tight_layout()
#     plt.savefig(f"{output_dir}/{filename}.png", 
#                 dpi=300, 
#                 bbox_inches='tight',
#                 transparent=True)   
#     plt.show()





Unique values in 'gender':
['Men' 'Women' 'Boys' 'Girls' 'Unisex']


Unique values in 'subCategory':
['Topwear' 'Bottomwear' 'Watches' 'Socks' 'Shoes' 'Belts' 'Flip Flops'
 'Bags' 'Innerwear' 'Sandal' 'Shoe Accessories' 'Fragrance' 'Jewellery'
 'Lips' 'Saree' 'Eyewear' 'Scarves' 'Dress' 'Loungewear and Nightwear'
 'Wallets' 'Apparel Set' 'Headwear' 'Mufflers' 'Skin Care' 'Makeup'
 'Free Gifts' 'Ties' 'Accessories' 'Nails' 'Beauty Accessories'
 'Water Bottle' 'Skin' 'Eyes' 'Bath and Body' 'Gloves'
 'Sports Accessories' 'Cufflinks' 'Sports Equipment' 'Stoles' 'Hair'
 'Perfumes' 'Home Furnishing' 'Umbrellas' 'Wristbands' 'Vouchers']


Unique values in 'baseColour':
['Navy Blue' 'Blue' 'Silver' 'Black' 'Grey' 'Green' 'Purple' 'White'
 'Beige' 'Brown' 'Bronze' 'Teal' 'Copper' 'Pink' 'Off White' 'Maroon'
 'Red' 'Khaki' 'Orange' 'Yellow' 'Charcoal' 'Gold' 'Steel' 'Tan' 'Multi'
 'Magenta' 'Lavender' 'Sea Green' 'Cream' 'Peach' 'Olive' 'Skin'
 'Burgundy' 'Coffee Brown' 'Grey Melange' 'Rust' 'Rose' 'Lime Green'
 'Mauve' 'Turquoise Blue' 'Metallic' 'Mustard' 'Taupe' 'Nude'
 'Mushroom Brown' 'Fluorescent Green']


Unique values in 'season':
['Fall' 'Summer' 'Winter' 'Spring']


Unique values in 'usage':
['Casual' 'Ethnic' 'Formal' 'Sports' 'Smart Casual' 'Travel' 'Party'
 'Home']

In [25]:
# num_rows = len(styles_df.columns) - 1

# fig, axes = plt.subplots(nrows=num_rows, ncols=1, figsize=(15, 5 * num_rows))


# axes_index = 0

# for column in styles_df.columns:
#     if column == 'id':  
#         continue

#     print(f"Unique values in '{column}':")
#     print(styles_df[column].unique())
#     print("\n")

#     sns.countplot(data=styles_df, x=column, order=styles_df[column].value_counts().index, ax=axes[axes_index])
#     axes[axes_index].set_title(f'Distribution of {column}')
#     axes[axes_index].set_xticklabels(axes[axes_index].get_xticklabels(), rotation=45)  # Rotate x labels if needed
#     axes[axes_index].set_ylabel('Count')

    
#     axes_index += 1

# plt.tight_layout()
# plt.show()
# plt.save

In [37]:
df = styles_df[:15000].copy()
df

,id,gender,subCategory,baseColour,season,usage
0,15970,Men,Topwear,Navy Blue,Fall,Casual
1,39386,Men,Bottomwear,Blue,Summer,Casual
2,59263,Women,Watches,Silver,Winter,Casual
3,21379,Men,Bottomwear,Black,Fall,Casual
4,53759,Men,Topwear,Grey,Summer,Casual
...,...,...,...,...,...,...
15108,55396,Women,Eyes,Black,Spring,Casual
15110,24557,Women,Topwear,Yellow,Fall,Ethnic
15111,10762,Unisex,Flip Flops,Yellow,Winter,Casual
15112,40012,Men,Eyewear,Brown,Winter,Casual


In [27]:

# image_dir = 'images'
# image_filenames = os.listdir(image_dir)
# image_filenames = [f for f in image_filenames if f.endswith(('jpg', 'jpeg', 'png'))]
# num_images = len(image_filenames)
# if num_images < 9:
#     print(f"Only {num_images} images found. Selecting all available images.")
#     selected_images = image_filenames  
# else:
#     selected_images = random.sample(image_filenames, 9)


# fig, axes = plt.subplots(3, 3, figsize=(12, 12))
# axes = axes.flatten()

# for ax, img_filename in zip(axes, selected_images):
    
#     img_path = os.path.join(image_dir, img_filename)
#     img = cv2.imread(img_path)
    
#     if img is None:
#         print(f"Image {img_filename} could not be loaded.")
#         ax.axis('off')  
#         ax.set_title(f"Failed to load")
#         continue  

#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     ax.imshow(img)
#     ax.set_title(img_filename)
#     ax.axis('off')

# # Show the plot
# plt.tight_layout()
# plt.show()

In [28]:
image_ids = df.pop('id')
image_ids = [f"images/{str(img_id)}.jpg" for img_id in image_ids]


In [29]:
def bin_values(x):
    x_vc = df[x].value_counts()
    x_other = x_vc[x_vc<50].index
    df.loc[df[x].isin(x_other),x] = 'Other'
    
    
for col in df.columns:
    bin_values(col)

In [30]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [31]:
IMAGE_DIMS = (180, 180, 3)

def load_image(imagePath):
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = preprocess_input(image)
    return image

In [32]:
image_data = []
for img_path in tqdm(image_ids[:15000]): 
    img = load_image(img_path)
    if img is not None:
        image_data.append(img)

100%|██████████| 15000/15000 [05:07<00:00, 48.77it/s]


In [34]:
image_data = np.array(image_data)

In [36]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def prepare_fashion_data(image_paths, category_labels, color_labels, test_size=0.2):
    """
    Prepare fashion data for multi-output classification
    
    Parameters:
    -----------
    image_paths : list
        List of image file paths
    category_labels : list or array
        Category labels corresponding to each image
    color_labels : list or array
        Color labels corresponding to each image
    test_size : float
        Proportion of data to use for testing
        
    Returns:
    --------
    dict
        Dictionary containing processed data and label binarizers
    """
    # Load images
    print("Loading and preprocessing images...")
    image_data = []
    valid_indices = []
    
    for idx, img_path in enumerate(tqdm(image_paths)):
        img = load_image(img_path)
        if img is not None:
            image_data.append(img)
            valid_indices.append(idx)
    
    # Convert to numpy array
    image_data = np.array(image_data)
    
    # Filter labels to match valid images
    category_labels = np.array(category_labels)[valid_indices]
    color_labels = np.array(color_labels)[valid_indices]
    
    # Initialize label binarizers
    category_lb = LabelBinarizer()
    color_lb = LabelBinarizer()
    
    # Transform labels
    category_onehot = category_lb.fit_transform(category_labels)
    color_onehot = color_lb.fit_transform(color_labels)
    
    # Create class names dictionary
    class_names_dict = {
        'category': category_lb.classes_.tolist(),
        'color': color_lb.classes_.tolist()
    }
    
    # Save class names
    import json
    with open('fashion_class_names.json', 'w') as f:
        json.dump(class_names_dict, f)
    
    # Split data
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    
    # Use category for stratification
    for train_idx, test_idx in sss.split(image_data, category_labels):
        trainX = image_data[train_idx]
        testX = image_data[test_idx]
        
        train_category = category_onehot[train_idx]
        test_category = category_onehot[test_idx]
        
        train_color = color_onehot[train_idx]
        test_color = color_onehot[test_idx]
    
    # Prepare dictionaries for model training
    trainY_dict = {
        'category_output': train_category,
        'color_output': train_color
    }
    
    testY_dict = {
        'category_output': test_category,
        'color_output': test_color
    }
    
    # Create label binarizers dictionary
    label_binarizers = {
        'category': category_lb,
        'color': color_lb
    }
    
    return {
        'trainX': trainX,
        'trainY_dict': trainY_dict,
        'testX': testX,
        'testY_dict': testY_dict,
        'label_binarizers': label_binarizers,
        'class_names_dict': class_names_dict
    }
    
    
def create_lightweight_branch(input_layer, num_classes, activation, name):
    """Create a simplified classification branch"""
    base_name = name.replace('_output', '')
    
    x = Dense(128, activation='relu', name=f'{name}_dense1')(input_layer)
    x = BatchNormalization(name=f'{name}_bn1')(x)
    x = Dropout(0.2, name=f'{name}_dropout1')(x)
    x = Dense(64, activation='relu', name=f'{name}_dense2')(x)
    output = Dense(num_classes, activation=activation, name=f'{name}_output')(x)
    return output

def build_optimized_model(width, height, num_classes_dict):
    """Build a lightweight multi-output model"""
    IMAGE_DIMS = (height, width, 3)
    
    base_model = MobileNetV2(
        weights='imagenet', 
        include_top=False, 
        input_shape=IMAGE_DIMS,
        alpha=0.75  
    )
    base_model.trainable = False
    
    inputs = Input(shape=IMAGE_DIMS, name='input_image')
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D(name='gap')(x)  
    
    outputs = {}
    for task_name, num_classes in num_classes_dict.items():
        output_name = f'{task_name}_output'
        outputs[output_name] = create_lightweight_branch(
            x, 
            num_classes=num_classes,
            activation='softmax',
            name=task_name
        )
    
    model = Model(
        inputs=inputs,
        outputs=outputs
    )
    
    return model

def train_optimized_model(trainX, trainY_dict, testX, testY_dict, epochs=40, batch_size=32):
    """Train the optimized multi-output fashion classification model"""
    print("\nTraining data shapes:")
    for name, labels in trainY_dict.items():
        print(f"{name}: {labels.shape}")
    
    num_classes_dict = {
        name.replace('_output', ''): labels.shape[1]
        for name, labels in trainY_dict.items()
    }
    
    model = build_optimized_model(180, 180, num_classes_dict)
    
    losses = {name: "categorical_crossentropy" for name in trainY_dict.keys()}
    metrics = {name: "accuracy" for name in trainY_dict.keys()}
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=losses, metrics=metrics)
    
    model.summary()
    
    history = model.fit(
        trainX,
        trainY_dict,
        validation_data=(testX, testY_dict),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    
    return model, history

def evaluate_models(model, testX, testY_dict, tflite_model=None, class_names_dict=None):
    """
    Evaluate the trained model and optionally its TFLite version
    
    Parameters:
    -----------
    model : keras.Model
        The trained multi-output Keras model
    testX : numpy.ndarray
        Test images
    testY_dict : dict
        Dictionary of test labels, with task names as keys and one-hot encoded labels as values
    tflite_model : bytes, optional
        TFLite model bytes for evaluation
    class_names_dict : dict, optional
        Dictionary mapping task names to lists of class names for detailed reporting
    
    Returns:
    --------
    dict
        Evaluation metrics for each model type and task
    """
    results = {'keras_model': {}, 'tflite_model': {}}
    
    # Evaluate Keras model
    print("\n===== Evaluating Keras Model =====")
    keras_eval = model.evaluate(testX, testY_dict, verbose=1)
    
    # Extract and format results
    loss_idx = 0
    for i, task_name in enumerate(testY_dict.keys()):
        accuracy_idx = loss_idx + 1
        results['keras_model'][task_name] = {
            'loss': keras_eval[loss_idx],
            'accuracy': keras_eval[accuracy_idx]
        }
        print(f"{task_name}: Loss = {keras_eval[loss_idx]:.4f}, Accuracy = {keras_eval[accuracy_idx]:.4f}")
        loss_idx += 2
    
    # Generate detailed classification metrics
    if class_names_dict:
        from sklearn.metrics import classification_report, confusion_matrix
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        # Get predictions
        predictions = model.predict(testX)
        
        for task_name in testY_dict.keys():
            # Convert one-hot encoded predictions and labels back to class indices
            pred_indices = np.argmax(predictions[task_name], axis=1)
            true_indices = np.argmax(testY_dict[task_name], axis=1)
            
            # Get class names for this task
            task_base_name = task_name.replace('_output', '')
            class_names = class_names_dict.get(task_base_name, None)
            
            print(f"\n===== Detailed Metrics for {task_base_name} =====")
            
            # Classification report
            print("\nClassification Report:")
            print(classification_report(
                true_indices, 
                pred_indices,
                target_names=class_names if class_names else None
            ))
            
            # Confusion matrix
            cm = confusion_matrix(true_indices, pred_indices)
            plt.figure(figsize=(10, 8))
            sns.heatmap(
                cm, 
                annot=True, 
                fmt="d", 
                cmap="Blues",
                xticklabels=class_names if class_names else "auto",
                yticklabels=class_names if class_names else "auto"
            )
            plt.title(f'Confusion Matrix for {task_base_name}')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            plt.tight_layout()
            plt.savefig(f'confusion_matrix_{task_base_name}.png')
            plt.close()
    
    # Evaluate TFLite model if provided
    if tflite_model is not None:
        print("\n===== Evaluating TFLite Model =====")
        
        # Create TFLite interpreter
        interpreter = tf.lite.Interpreter(model_content=tflite_model)
        interpreter.allocate_tensors()
        
        # Get input and output details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Prepare for accuracy calculation
        task_correct_predictions = {task: 0 for task in testY_dict.keys()}
        total_samples = testX.shape[0]
        
        # Process batch by batch to avoid memory issues
        batch_size = 32
        for i in range(0, total_samples, batch_size):
            batch_end = min(i + batch_size, total_samples)
            batch_x = testX[i:batch_end]
            
            # Process each sample in the batch
            for j in range(batch_x.shape[0]):
                sample = batch_x[j:j+1].astype(np.float32)
                
                # Set input tensor
                interpreter.set_tensor(input_details[0]['index'], sample)
                
                # Run inference
                interpreter.invoke()
                
                # Get outputs
                for k, details in enumerate(output_details):
                    task_name = list(testY_dict.keys())[k]
                    task_base_name = task_name.replace('_output', '')
                    
                    output_data = interpreter.get_tensor(details['index'])
                    pred_class = np.argmax(output_data[0])
                    true_class = np.argmax(testY_dict[task_name][i+j])
                    
                    if pred_class == true_class:
                        task_correct_predictions[task_name] += 1
        
        # Calculate and display accuracies
        for task_name in testY_dict.keys():
            accuracy = task_correct_predictions[task_name] / total_samples
            results['tflite_model'][task_name] = {'accuracy': accuracy}
            print(f"{task_name}: TFLite Accuracy = {accuracy:.4f}")
    
    return results


def convert_to_tflite(model, quantize=True):
    """Convert model to TFLite format with optional quantization"""
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    if quantize:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]
    
    tflite_model = converter.convert()
    
    # Print size information
    print(f"TFLite model size: {len(tflite_model) / (1024 * 1024):.2f} MB")
    
    return tflite_model

def run_optimized_pipeline(
    data_prep_results,
    epochs=40,
    batch_size=32,
    verbose=True):
    """Run the optimized model training pipeline using previously prepared data"""
    trainX = data_prep_results['trainX']
    trainY_dict = data_prep_results['trainY_dict']
    testX = data_prep_results['testX']
    testY_dict = data_prep_results['testY_dict']
    class_names_dict = data_prep_results.get('class_names_dict', None)
    
    print("\nStarting optimized model training...")
    model, history = train_optimized_model(
        trainX,
        trainY_dict,
        testX,
        testY_dict,
        epochs=epochs,
        batch_size=batch_size
    )
    
    # Convert to TFLite
    tflite_model = convert_to_tflite(model, quantize=True)
    
    # Save TFLite model
    with open('fashion_classifier_optimized.tflite', 'wb') as f:
        f.write(tflite_model)
    
    # Save regular model with optimization
    model.save('fashion_classifier_optimized.h5', save_format='h5')
    
    # Try to save in TensorFlow SavedModel format which is more efficient
    model.save('fashion_classifier_optimized', save_format='tf')
    
    # Run evaluation
    print("\nEvaluating models...")
    eval_results = evaluate_models(
        model, 
        testX, 
        testY_dict, 
        tflite_model=tflite_model,
        class_names_dict=class_names_dict
    )
    
    return {
        'model': model,
        'tflite_model': tflite_model,
        'history': history,
        'evaluation': eval_results,
        'class_names_dict': class_names_dict
    }

def load_class_names():
    """Load previously saved class names dictionary"""
    import json
    try:
        with open('class_names.json', 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Class names file not found. Returning empty dictionary.")
        return {}

# Example of how to load and use the class names for evaluation later
def evaluate_saved_model(model_path='fashion_classifier_optimized.h5', test_data=None):
    """Evaluate a previously saved model"""
    from tensorflow.keras.models import load_model
    
    # Load the model
    model = load_model(model_path)
    
    # Load class names
    class_names_dict = load_class_names()
    
    # Load test data if not provided
    if test_data is None:
        # You'd need logic here to load your test data
        pass
    
    # Run evaluation
    eval_results = evaluate_models(
        model,
        test_data['testX'],
        test_data['testY_dict'],
        class_names_dict=class_names_dict
    )
    
    return eval_results


prepared_data = prepare_fashion_data(
    image_ids[:15000],  # Your image paths
    category_labels,    # Your category labels
    color_labels        # Your color labels
)

optimized_results = run_optimized_pipeline(prepared_data, epochs=40)

NameError: name 'category_labels' is not defined

In [1]:
# def evaluate_model(model, testX, testY_dict):
#     print("\nEvaluating the model on test data...")
#     print(f"testX shape: {testX.shape}")
    
#     # Show output shapes for debugging
#     for name, y in testY_dict.items():
#         print(f"{name} shape: {y.shape}")
    
#     # Get predictions
#     print("\nGenerating predictions...")
#     predictions = model.predict(testX, verbose=1)
    
#     # Calculate metrics manually
#     results = {}
#     print("\nIndividual Output Accuracies:")
#     for output_name in model.output_names:
#         if output_name in predictions and output_name in testY_dict:
#             y_true = testY_dict[output_name]
#             y_pred = predictions[output_name]
            
#             # Calculate categorical accuracy
#             accuracy = np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))
#             results[f"{output_name}_accuracy"] = accuracy
#             print(f"{output_name} accuracy: {accuracy:.4f}")
            
#             # Calculate top-3 accuracy for categories with many classes
#             if y_true.shape[1] > 10:  # For outputs with many classes
#                 top3_accuracy = calculate_top_k_accuracy(y_true, y_pred, k=3)
#                 results[f"{output_name}_top3_accuracy"] = top3_accuracy
#                 print(f"{output_name} top-3 accuracy: {top3_accuracy:.4f}")
    
#     return results

# def calculate_top_k_accuracy(y_true, y_pred, k=3):
#     """Calculate top-k accuracy for multi-class classification"""
#     true_classes = np.argmax(y_true, axis=1)
#     pred_classes = np.argsort(y_pred, axis=1)[:, -k:]  # Get top k predictions
    
#     # Check if true class is in top k predictions for each sample
#     correct = 0
#     for i, true_class in enumerate(true_classes):
#         if true_class in pred_classes[i]:
#             correct += 1
    
#     return correct / len(true_classes)
# evaluate_model(model, testX, testY_dict)

In [2]:
# def evaluate_model(model, testX, testY_dict, label_binarizers=None, num_classes_dict=None):
#     print("\nEvaluating the model on test data...")
#     print(f"testX shape: {testX.shape}")
    
#     # Import necessary libraries if not already imported
#     from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
    
#     # Show output shapes for debugging
#     for name, y in testY_dict.items():
#         print(f"{name} shape: {y.shape}")
    
#     # Get predictions
#     print("\nGenerating predictions...")
#     predictions = model.predict(testX, verbose=1)
    
#     # Calculate metrics manually
#     results = {}
#     print("\nPerformance Metrics:")
    
#     for output_name in model.output_names:
#         if output_name in predictions and output_name in testY_dict:
#             print(f"\n----- {output_name} Metrics -----")
#             y_true = testY_dict[output_name]
#             y_pred = predictions[output_name]
            
#             # Get the attribute name without the "_output" suffix
#             attr_name = output_name.replace('_output', '')
            
#             # Convert to class indices
#             y_true_classes = np.argmax(y_true, axis=1)
#             y_pred_classes = np.argmax(y_pred, axis=1)
            
#             # Get class names if label_binarizers is available
#             class_names = None
#             if label_binarizers and attr_name in label_binarizers:
#                 try:
#                     class_names = label_binarizers[attr_name].classes_
#                     print(f"\nClass mapping for {attr_name}:")
#                     for idx, name in enumerate(class_names):
#                         print(f"Class {idx}: {name}")
#                 except AttributeError:
#                     print(f"Warning: Could not access classes_ attribute in label_binarizer for {attr_name}")
#                     class_names = None
            
#             # Calculate accuracy
#             accuracy = np.mean(y_true_classes == y_pred_classes)
#             results[f"{output_name}_accuracy"] = accuracy
#             print(f"Accuracy: {accuracy:.4f}")
            
#             # Calculate top-3 accuracy for categories with many classes
#             if y_true.shape[1] > 10:  # For outputs with many classes
#                 top3_accuracy = calculate_top_k_accuracy(y_true, y_pred, k=3)
#                 results[f"{output_name}_top3_accuracy"] = top3_accuracy
#                 print(f"Top-3 Accuracy: {top3_accuracy:.4f}")
            
#             # Calculate precision, recall, and F1 (macro-averaged)
#             precision = precision_score(y_true_classes, y_pred_classes, average='macro', zero_division=0)
#             recall = recall_score(y_true_classes, y_pred_classes, average='macro', zero_division=0)
#             f1 = f1_score(y_true_classes, y_pred_classes, average='macro', zero_division=0)
            
#             results[f"{output_name}_precision"] = precision
#             results[f"{output_name}_recall"] = recall
#             results[f"{output_name}_f1"] = f1
            
#             print(f"Precision (macro): {precision:.4f}")
#             print(f"Recall (macro): {recall:.4f}")
#             print(f"F1 Score (macro): {f1:.4f}")
            
#             # Calculate class-wise metrics for detailed analysis
#             class_precision = precision_score(y_true_classes, y_pred_classes, average=None, zero_division=0)
#             class_recall = recall_score(y_true_classes, y_pred_classes, average=None, zero_division=0)
#             class_f1 = f1_score(y_true_classes, y_pred_classes, average=None, zero_division=0)
            
#             # Find top and bottom performing classes
#             top_classes_idx = np.argsort(class_f1)[-3:][::-1]  # Top 3 classes by F1
#             bottom_classes_idx = np.argsort(class_f1)[:3]      # Bottom 3 classes by F1
            
#             print("\nTop 3 Classes by F1 Score:")
#             for idx in top_classes_idx:
#                 if class_names is not None and idx < len(class_names):
#                     class_label = class_names[idx]
#                 else:
#                     class_label = f"Class {idx}"
#                 print(f"{class_label}: F1={class_f1[idx]:.4f}, Precision={class_precision[idx]:.4f}, Recall={class_recall[idx]:.4f}")
            
#             print("\nBottom 3 Classes by F1 Score:")
#             for idx in bottom_classes_idx:
#                 if class_names is not None and idx < len(class_names):
#                     class_label = class_names[idx]
#                 else:
#                     class_label = f"Class {idx}"
#                 print(f"{class_label}: F1={class_f1[idx]:.4f}, Precision={class_precision[idx]:.4f}, Recall={class_recall[idx]:.4f}")
            
#             # Generate confusion matrix
#             cm = confusion_matrix(y_true_classes, y_pred_classes)
            
#             # If there are many classes, calculate a simplified confusion matrix
#             # showing just the most confused pairs
#             if y_true.shape[1] > 10:
#                 print("\nTop Misclassifications:")
#                 # Find pairs with high misclassification rates
#                 np.fill_diagonal(cm, 0)  # Ignore the diagonal (correct predictions)
#                 if np.sum(cm) > 0:  # Only proceed if there are misclassifications
#                     total_misclassified = np.sum(cm)
                    
#                     # Get top 5 misclassification pairs
#                     flat_indices = np.argsort(cm.flatten())[-5:]
#                     for idx in flat_indices:
#                         true_class = idx // cm.shape[1]
#                         pred_class = idx % cm.shape[1]
                        
#                         # Check bounds for class names
#                         if class_names is not None:
#                             true_name = class_names[true_class] if true_class < len(class_names) else f"Class {true_class}"
#                             pred_name = class_names[pred_class] if pred_class < len(class_names) else f"Class {pred_class}"
#                         else:
#                             true_name = f"Class {true_class}"
#                             pred_name = f"Class {pred_class}"
                        
#                         misclass_rate = cm[true_class, pred_class] / total_misclassified
#                         print(f"True: {true_name} → Predicted: {pred_name}: {cm[true_class, pred_class]} instances ({misclass_rate:.1%} of all errors)")
#                 else:
#                     print("No misclassifications found (perfect predictions on test set)")
            
#             # Store confusion matrix and class names
#             results[f"{output_name}_confusion_matrix"] = cm
#             if class_names is not None:
#                 results[f"{output_name}_class_names"] = class_names
            
#     return results
# def calculate_top_k_accuracy(y_true, y_pred, k=3):
#     """Calculate top-k accuracy for multi-class classification"""
#     true_classes = np.argmax(y_true, axis=1)
#     pred_classes = np.argsort(y_pred, axis=1)[:, -k:]  # Get top k predictions
    
#     # Check if true class is in top k predictions for each sample
#     correct = 0
#     for i, true_class in enumerate(true_classes):
#         if true_class in pred_classes[i]:
#             correct += 1
    
#     return correct / len(true_classes)

# def plot_confusion_matrix(cm, class_names=None, figsize=(10, 8), title='Confusion Matrix'):
#     """Plot a confusion matrix with better visualization for many classes"""
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     import numpy as np
    
#     # If there are many classes, plot a heatmap only
#     if cm.shape[0] > 10:
#         plt.figure(figsize=figsize)
#         # Use logarithmic scale for better visualization when classes are imbalanced
#         sns.heatmap(cm, cmap='Blues', norm=LogNorm(vmin=0.1, vmax=cm.max()))
#         plt.title(title)
#         plt.ylabel('True Label')
#         plt.xlabel('Predicted Label')
#         plt.show()
#     else:
#         # For fewer classes, we can show actual numbers and class names
#         plt.figure(figsize=figsize)
        
#         if class_names is None:
#             class_names = [str(i) for i in range(cm.shape[0])]
            
#         # Calculate percentages (normalize by row)
#         cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
#         # Create a custom annotation function to show counts and percentages
#         annot = np.empty_like(cm, dtype=object)
#         for i in range(cm.shape[0]):
#             for j in range(cm.shape[1]):
#                 annot[i, j] = f"{cm[i, j]}\n({cm_normalized[i, j]:.1%})"
        
#         sns.heatmap(cm, annot=annot, fmt="", cmap="Blues", 
#                     xticklabels=class_names, 
#                     yticklabels=class_names)
#         plt.ylabel('True Label')
#         plt.xlabel('Predicted Label')
#         plt.title(title)
#         plt.tight_layout()
#         plt.show()


# def plot_and_save_confusion_matrices(results, output_dir='confusion_matrices', figsize=(12, 10)):
#     """
#     Plot and save confusion matrices for all outputs
    
#     Parameters:
#     results: Dictionary containing evaluation results with confusion matrices
#     output_dir: Directory to save the plots (will be created if it doesn't exist)
#     figsize: Size of the figure for each confusion matrix
#     """
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     import numpy as np
#     import os
#     from matplotlib.colors import LogNorm
    
#     # Create output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Find all confusion matrices in results
#     for key in results:
#         if key.endswith('_confusion_matrix'):
#             output_name = key.replace('_confusion_matrix', '')
#             cm = results[key]
            
#             # Get class names if available
#             class_names = None
#             if f"{output_name}_class_names" in results:
#                 class_names = results[f"{output_name}_class_names"]
            
#             # Create figure
#             plt.figure(figsize=figsize)
            
#             # Decide on visualization approach based on number of classes
#             if cm.shape[0] > 10:
#                 # For many classes, use a heatmap without annotations
#                 sns.heatmap(cm, cmap='Blues', norm=LogNorm(vmin=0.1, vmax=cm.max()))
#                 plt.title(f"{output_name} Confusion Matrix")
#                 plt.ylabel('True Label')
#                 plt.xlabel('Predicted Label')
#             else:
#                 # For fewer classes, show counts and percentages
#                 # Calculate percentages (normalize by row)
#                 cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
                
#                 # Create a custom annotation function to show counts and percentages
#                 annot = np.empty_like(cm, dtype=object)
#                 for i in range(cm.shape[0]):
#                     for j in range(cm.shape[1]):
#                         annot[i, j] = f"{cm[i, j]}\n({cm_normalized[i, j]:.1%})"
                
#                 # Get tick labels
#                 if class_names is not None:
#                     xticklabels = class_names
#                     yticklabels = class_names
#                 else:
#                     xticklabels = list(range(cm.shape[1]))
#                     yticklabels = list(range(cm.shape[0]))
                
#                 sns.heatmap(cm, annot=annot, fmt="", cmap="Blues", 
#                            xticklabels=xticklabels, 
#                            yticklabels=yticklabels)
#                 plt.ylabel('True Label')
#                 plt.xlabel('Predicted Label')
#                 plt.title(f"{output_name} Confusion Matrix")
            
#             plt.tight_layout()
            
#             # Save the figure
#             filename = os.path.join(output_dir, f"{output_name}_confusion_matrix.png")
#             plt.savefig(filename, dpi=300, bbox_inches='tight')
#             print(f"Saved {filename}")
            
#             # Close the figure to free memory
#             plt.close()
            
#     print(f"All confusion matrices saved to {output_dir}/")

In [3]:
# plot_and_save_confusion_matrices(results, output_dir='model_evaluation/confusion_matrices')

In [4]:
# results = evaluate_model(
#     model, 
#     testX, 
#     testY_dict,
#     label_binarizers=label_binarizers,
#     num_classes_dict=num_classes_dict
# )

In [5]:
# data = advanced_data_preparation(image_data, df, sample_size=13200, test_size=0.2, random_state=42, z_threshold=3)
# testX = data['testX']
# testY_dict = data['testY_dict']


In [6]:
# def evaluate_model(model, testX, testY_dict):
#     print("\nEvaluating the model on test data...")
#     print(f"testX shape: {testX.shape}")

#     ordered_testY = []
#     for name in model.output_names:
#         y = testY_dict.get(name)
#         if y is None:
#             raise ValueError(f"Missing output: {name} in testY_dict")
#         print(f"{name} shape: {y.shape}")
#         ordered_testY.append(y)

#     # Now evaluate
#     results = model.evaluate(testX, ordered_testY, verbose=1)
#     metric_names = model.metrics_names

#     print("\nEvaluation Metrics:")
#     for name, value in zip(metric_names, results):
#         print(f"{name}: {value:.4f}")

#     return dict(zip(metric_names, results))


In [19]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer, RobustScaler
from scipy import stats
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def advanced_data_preparation(
    image_data, 
    df, 
    sample_size=None, 
    test_size=0.2, 
    random_state=42, 
    z_threshold=3,
    verbose=True
):
    """
    Comprehensive data preparation function with:
    - Sampling 
    - Outlier removal
    - Label binarization
    - Stratified splitting
    """
    if sample_size is None:
        sample_size = len(df)
    
    image_data_sampled = image_data[:sample_size]
    
    if verbose:
        print("\nOriginal DataFrame columns:", df.columns)
        for column in ['subCategory', 'gender', 'baseColour', 'season', 'usage']:
            unique_values = df[column].nunique()
            print(f"\nUnique values in {column}: {unique_values}")
            print(f"Values: {sorted(df[column].unique())}")
    
    
    label_mapping = {
        'subCategory': 'subCategory',
        'gender': 'gender',
        'baseColour': 'color',
        'season': 'season',
        'usage': 'usage'
    }
    
    label_binarizers = {
        output_name: LabelBinarizer()
        for output_name in label_mapping.values()
    }
    
    labels_dict = {}
    for input_name, output_name in label_mapping.items():
        labels_dict[f'{output_name}_output'] = label_binarizers[output_name].fit_transform(
            np.array(df[input_name].values[:sample_size])
        )
    
    def remove_outliers(data, labels_dict, z_threshold=3):
        """Remove outliers using z-score method across multiple labels"""
        total_samples = len(data)
        keep_mask = np.ones(total_samples, dtype=bool)
        
        if verbose:
            print("\nInitial Shapes:")
            print(f"Data shape: {data.shape}")
            for name, labels in labels_dict.items():
                print(f"{name} shape: {labels.shape}")
        
        for label_name, labels in labels_dict.items():
            if verbose:
                print(f"\nProcessing {label_name}:")
                print(f"Initial samples: {np.sum(keep_mask)}")
            
            label_indices = np.argmax(labels, axis=1)
            z_scores = np.abs(stats.zscore(label_indices))
            current_mask = z_scores < z_threshold
            keep_mask = keep_mask & current_mask
            
            if verbose:
                print(f"Samples after filtering: {np.sum(keep_mask)}")
        
        clean_data = data[keep_mask]
        clean_labels_dict = {
            key: labels[keep_mask] for key, labels in labels_dict.items()
        }
        
        if verbose:
            print("\nAfter removing outliers:")
            print(f"Clean data shape: {clean_data.shape}")
            for name, labels in clean_labels_dict.items():
                print(f"{name} shape: {labels.shape}")
        
        return clean_data, clean_labels_dict
    
    clean_data, clean_labels_dict = remove_outliers(
        image_data_sampled, 
        labels_dict, 
        z_threshold=z_threshold
    )
    
    stratified_split = StratifiedShuffleSplit(
        n_splits=1, 
        test_size=test_size, 
        random_state=random_state
    )
    
    primary_label = clean_labels_dict['subCategory_output']
    primary_label_indices = np.argmax(primary_label, axis=1)
    
    for train_index, test_index in stratified_split.split(clean_data, primary_label_indices):
        trainX = clean_data[train_index]
        testX = clean_data[test_index]
        trainY_dict = {
            key: labels[train_index] for key, labels in clean_labels_dict.items()
        }
        testY_dict = {
            key: labels[test_index] for key, labels in clean_labels_dict.items()
        }
    
    num_classes_dict = {
        key.replace('_output', ''): labels.shape[1] 
        for key, labels in trainY_dict.items()
    }
    
    if verbose:
        print("\nFinal training shapes:")
        print(f"trainX shape: {trainX.shape}")
        for name, labels in trainY_dict.items():
            print(f"{name} shape: {labels.shape}")
        print("\nNumber of Classes:")
        for name, num_classes in num_classes_dict.items():
            print(f"{name}: {num_classes} classes")
    
    return {
        'trainX': trainX,
        'trainY_dict': trainY_dict,
        'testX': testX,
        'testY_dict': testY_dict,
        'num_classes_dict': num_classes_dict,
        'label_binarizers': label_binarizers
    }

def create_branch(input_layer, num_classes, activation, name):
    """Create a classification branch with proper naming"""
    base_name = name.replace('_output', '')
    
    x = Dense(512, activation='relu', name=f'{name}_dense1')(input_layer)
    x = Dense(256, activation='relu', name=f'{name}_dense2')(x)
    x = Dense(32, activation='relu', name=f'{name}_dense3')(x)
    x = BatchNormalization(name=f'{name}_bn1')(x)
    x = Dense(128, activation='relu', name=f'{name}_dense4')(x)
    x = Dense(64, activation='relu', name=f'{name}_dense5')(x)
    x = Dense(32, activation='relu', name=f'{name}_dense6')(x)
    x = BatchNormalization(name=f'{name}_bn2')(x)
    x = Dense(128, activation='relu', name=f'{name}_dense7')(x)
    x = Dense(64, activation='relu', name=f'{name}_dense8')(x)
    x = Dense(32, activation='relu', name=f'{name}_dense9')(x)
    output = Dense(num_classes, activation=activation, name=f'{name}_output')(x)
    return output

def build_model(width, height, num_classes_dict):
    """Build the multi-output model"""
    IMAGE_DIMS = (height, width, 3)
    
    base_model = ResNet50(weights='imagenet', 
                         include_top=False, 
                         input_shape=IMAGE_DIMS)
    base_model.trainable = False
    
    inputs = Input(shape=IMAGE_DIMS, name='input_image')
    x = base_model(inputs, training=False)
    x = Flatten(name='flatten')(x)
    
    outputs = {}
    for task_name, num_classes in num_classes_dict.items():
        output_name = f'{task_name}_output'
        outputs[output_name] = create_branch(
            x, 
            num_classes=num_classes,
            activation='softmax',
            name=task_name
        )
    
    model = Model(
        inputs=inputs,
        outputs=outputs
    )
    print("\nModel output shapes with names:")
    for layer in model.output_names:
        print(f"{layer}: {model.get_layer(layer).output.shape}")
    return model

def train_model(trainX, trainY_dict, testX, testY_dict, epochs=40, batch_size=32):
    """Train the multi-output fashion classification model"""
    # Input validation
    # assert trainX.shape[0] == list(trainY_dict.values())[0].shape[0], "Inconsistent training data"
    # assert testX.shape[0] == list(testY_dict.values())[0].shape[0], "Inconsistent test data"

    print("\nTraining data shapes:")
    for name, labels in trainY_dict.items():
        print(f"{name}: {labels.shape}")
    
    # Get number of classes for each task from the training data
    num_classes_dict = {
        name.replace('_output', ''): labels.shape[1]
        for name, labels in trainY_dict.items()
    }
    print("\nTask order:")
    for task_name in num_classes_dict.keys():
        print(f"{task_name}: {num_classes_dict[task_name]} classes")
    
    # Build model
    model = build_model(180, 180, num_classes_dict)
    
    
    losses = {name: "categorical_crossentropy" for name in trainY_dict.keys()}
    metrics = {name: "accuracy" for name in trainY_dict.keys()}
    

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=losses, metrics=metrics)
    
   
    model.summary()
    
    
    print("\nInput shapes:")
    print(f"trainX shape: {trainX.shape}")
    for name, y in trainY_dict.items():
        print(f"{name} shape: {y.shape}")
    

    history = model.fit(
        trainX,
        trainY_dict,
        validation_data=(testX, testY_dict),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    
    return model, history


def run_fashion_classification_pipeline(
    image_data,
    df,
    sample_size=15000,
    test_size=0.2,
    random_state=42,
    z_threshold=3,
    epochs=40,
    batch_size=32,
    verbose=True
):
    """Complete pipeline for fashion classification"""
    print("Input validation:")
    print(f"Image data shape: {image_data.shape}")
    print(f"DataFrame shape: {df.shape}")
    print(f"Sample size requested: {sample_size}")
    
    print("Starting data preparation...")
    data_prep_results = advanced_data_preparation(
        image_data,
        df,
        sample_size=sample_size,
        test_size=test_size,
        random_state=random_state,
        z_threshold=z_threshold,
        verbose=verbose
    )
    
   
    trainX = data_prep_results['trainX']
    trainY_dict = data_prep_results['trainY_dict']
    testX = data_prep_results['testX']
    testY_dict = data_prep_results['testY_dict']

    print("\nPrepared data shapes:")
    print(f"trainX: {trainX.shape}")
    print(f"testX: {testX.shape}")
    for name, labels in trainY_dict.items():
        print(f"train {name}: {labels.shape}")
    for name, labels in testY_dict.items():
        print(f"test {name}: {labels.shape}")
    
    
    print("\nStarting model training...")
    model, history = train_model(
        trainX,
        trainY_dict,
        testX,
        testY_dict,
        epochs=epochs,
        batch_size=batch_size
    )
    
    return {
        'model': model,
        'history': history,
        'data_prep_results': data_prep_results
    }
    

In [20]:
def create_branch(input_layer, num_classes, activation, name):
    """Create a classification branch with proper naming"""
    base_name = name.replace('_output', '')
    
    x = Dense(512, activation='relu', name=f'{name}_dense1')(input_layer)
    x = Dense(256, activation='relu', name=f'{name}_dense2')(x)
    x = Dense(32, activation='relu', name=f'{name}_dense3')(x)
    x = BatchNormalization(name=f'{name}_bn1')(x)
    x = Dense(128, activation='relu', name=f'{name}_dense4')(x)
    x = Dense(64, activation='relu', name=f'{name}_dense5')(x)
    x = Dense(32, activation='relu', name=f'{name}_dense6')(x)
    x = BatchNormalization(name=f'{name}_bn2')(x)
    x = Dense(128, activation='relu', name=f'{name}_dense7')(x)
    x = Dense(64, activation='relu', name=f'{name}_dense8')(x)
    x = Dense(32, activation='relu', name=f'{name}_dense9')(x)
    output = Dense(num_classes, activation=activation, name=f'{name}_output')(x)
    return output

def build_model(width, height, num_classes_dict):
    """Build the multi-output model"""
    IMAGE_DIMS = (height, width, 3)
    
    base_model = ResNet50(weights='imagenet', 
                         include_top=False, 
                         input_shape=IMAGE_DIMS)
    base_model.trainable = False
    
    inputs = Input(shape=IMAGE_DIMS, name='input_image')
    x = base_model(inputs, training=False)
    x = Flatten(name='flatten')(x)
    
    outputs = {}
    for task_name, num_classes in num_classes_dict.items():
        output_name = f'{task_name}_output'
        outputs[output_name] = create_branch(
            x, 
            num_classes=num_classes,
            activation='softmax',
            name=task_name
        )
    
    model = Model(
        inputs=inputs,
        outputs=outputs
    )
    print("\nModel output shapes with names:")
    for layer in model.output_names:
        print(f"{layer}: {model.get_layer(layer).output.shape}")
    return model

def train_model(trainX, trainY_dict, testX, testY_dict, epochs=40, batch_size=32):
    """Train the multi-output fashion classification model"""
    # Input validation
    # assert trainX.shape[0] == list(trainY_dict.values())[0].shape[0], "Inconsistent training data"
    # assert testX.shape[0] == list(testY_dict.values())[0].shape[0], "Inconsistent test data"

    print("\nTraining data shapes:")
    for name, labels in trainY_dict.items():
        print(f"{name}: {labels.shape}")
    
    # Get number of classes for each task from the training data
    num_classes_dict = {
        name.replace('_output', ''): labels.shape[1]
        for name, labels in trainY_dict.items()
    }
    print("\nTask order:")
    for task_name in num_classes_dict.keys():
        print(f"{task_name}: {num_classes_dict[task_name]} classes")
    
    # Build model
    model = build_model(180, 180, num_classes_dict)
    
    
    losses = {name: "categorical_crossentropy" for name in trainY_dict.keys()}
    metrics = {name: "accuracy" for name in trainY_dict.keys()}
    

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=losses, metrics=metrics)
    
   
    model.summary()
    
    
    print("\nInput shapes:")
    print(f"trainX shape: {trainX.shape}")
    for name, y in trainY_dict.items():
        print(f"{name} shape: {y.shape}")
    

    history = model.fit(
        trainX,
        trainY_dict,
        validation_data=(testX, testY_dict),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    
    return model, history

def save_fashion_model(model, label_binarizers, num_classes_dict, save_path='fashion_model'):
    """
    Save the fashion classification model and all necessary components
    
    Args:
        model: trained Keras model
        label_binarizers: dictionary of fitted LabelBinarizer objects
        num_classes_dict: dictionary containing number of classes for each category
        save_path: base path for saving model files
    """

    os.makedirs(save_path, exist_ok=True)
    
   
    model.save(os.path.join(save_path, 'model.h5'))
    
   
    with open(os.path.join(save_path, 'label_binarizers.pkl'), 'wb') as f:
        pickle.dump(label_binarizers, f)
    

    with open(os.path.join(save_path, 'num_classes.json'), 'w') as f:
        json.dump(num_classes_dict, f)
    
    print(f"Model and components saved to {save_path}")

In [ ]:
# results = run_fashion_classification_pipeline(
#     image_data,
#     df,
#     sample_size=13200,  
#     epochs=10,
#     batch_size=32,
#     verbose=True
# )


# model = results['model']
# history = results['history']
# data_prep_results = results['data_prep_results']

In [21]:
import os
import json
import pickle
from tensorflow.keras.models import load_model
import numpy as np

def save_fashion_model(model, label_binarizers, num_classes_dict, save_path='fashion_model'):
    """
    Save the fashion classification model and all necessary components
    
    Args:
        model: trained Keras model
        label_binarizers: dictionary of fitted LabelBinarizer objects
        num_classes_dict: dictionary containing number of classes for each category
        save_path: base path for saving model files
    """

    os.makedirs(save_path, exist_ok=True)
    
   
    model.save(os.path.join(save_path, 'model.h5'))
    
   
    with open(os.path.join(save_path, 'label_binarizers.pkl'), 'wb') as f:
        pickle.dump(label_binarizers, f)
    

    with open(os.path.join(save_path, 'num_classes.json'), 'w') as f:
        json.dump(num_classes_dict, f)
    
    print(f"Model and components saved to {save_path}")

def load_fashion_model(load_path='fashion_model'):
    """
    Load the fashion classification model and all components
    
    Args:
        load_path: path where model files are saved
        
    Returns:
        model: loaded Keras model
        label_binarizers: dictionary of fitted LabelBinarizer objects
        num_classes_dict: dictionary containing number of classes for each category
    """
    
    model = load_model(os.path.join(load_path, 'model.h5'))
    
    
    with open(os.path.join(load_path, 'label_binarizers.pkl'), 'rb') as f:
        label_binarizers = pickle.load(f)
    
    
    with open(os.path.join(load_path, 'num_classes.json'), 'r') as f:
        num_classes_dict = json.load(f)
    
    print(f"Model and components loaded from {load_path}")
    return model, label_binarizers, num_classes_dict

def predict_fashion_item(model, label_binarizers, image_array):
    """
    Make predictions for a single fashion item
    
    Args:
        model: loaded Keras model
        label_binarizers: dictionary of fitted LabelBinarizer objects
        image_array: preprocessed image array of shape (180, 180, 3)
        
    Returns:
        predictions_dict: dictionary containing predictions for each category
    """
    
    if len(image_array.shape) == 3:
        image_array = np.expand_dims(image_array, axis=0)
    
    
    predictions = model.predict(image_array)
   
  
    if not isinstance(predictions, dict):
        raise ValueError("Model output is not a dictionary. Ensure the model is set up to output a dictionary.")
    predictions_dict = {}

    for category, lb in label_binarizers.items():
        prediction_key = f"{category}_output"
        
        
        if prediction_key not in predictions:
            raise ValueError(f"Missing prediction for category: {category}")
        
       
        pred = predictions[prediction_key]
        pred_class = np.argmax(pred)
        
        predictions_dict[category] = {
            'label': lb.classes_[pred_class],
            'confidence': float(pred[0][pred_class])
        }
    
    

    return predictions_dict

In [24]:
# def load_fashion_model(load_path='fashion_model'):
    
    
#     model = load_model(os.path.join(load_path, 'model.h5'))
    
    
#     with open(os.path.join(load_path, 'label_binarizers.pkl'), 'rb') as f:
#         label_binarizers = pickle.load(f)
    
    
#     with open(os.path.join(load_path, 'num_classes.json'), 'r') as f:
#         num_classes_dict = json.load(f)
    
#     print(f"Model and components loaded from {load_path}")
#     return model, label_binarizers, num_classes_dict

# model, label_binarizers, num_classes_dict = load_fashion_model(load_path="fashion_model")

# def preprocess_image(image_path, target_size=(180, 180)):
#     """
#     Preprocess an image for prediction.
#     Args:
#         image_path: Path to the image file.
#         target_size: Target size for resizing (default is 180x180).
#     Returns:
#         Preprocessed image array with batch dimension.
#     """
#     img = load_img(image_path, target_size=target_size)  
#     img_array = img_to_array(img) 
#     img_array = img_array / 255.0  
#     img_array = np.expand_dims(img_array, axis=0) 
#     return img_array

# IMAGE_DIMS = (180, 180, 3)
# def load_image(imagePath):
#     image = cv2.imread(imagePath)
#     image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = preprocess_input(image)
#     return image

# def predict_fashion_item(model, label_binarizers, image_array):
    
#     if len(image_array.shape) == 3:
#         image_array = np.expand_dims(image_array, axis=0)
    
    
#     predictions = model.predict(image_array)
   
  
#     if not isinstance(predictions, dict):
#         raise ValueError("Model output is not a dictionary. Ensure the model is set up to output a dictionary.")
#     predictions_dict = {}

#     for category, lb in label_binarizers.items():
#         prediction_key = f"{category}_output"
        
        
#         if prediction_key not in predictions:
#             raise ValueError(f"Missing prediction for category: {category}")
        
       
#         pred = predictions[prediction_key]
#         pred_class = np.argmax(pred)
        
#         predictions_dict[category] = {
#             'label': lb.classes_[pred_class],
#             'confidence': float(pred[0][pred_class])
#         }
    
    

#     return predictions_dict

# model, label_binarizers, num_classes_dict = load_fashion_model('fashion_model')

# image__path=r'images1\a.jpg'


# image = load_image(image__path)
# predictions = predict_fashion_item(model, label_binarizers, image)

# for category, pred in predictions.items():
#     print(f"{category}: {pred['label']} (confidence: {pred['confidence']:.2f})")

In [ ]:

# save_fashion_model(
#     model=results['model'],
#     label_binarizers=data_prep_results['label_binarizers'],
#     num_classes_dict=data_prep_results['num_classes_dict'],
#     save_path='fashion_model'
# )

In [25]:

model, label_binarizers, num_classes_dict = load_fashion_model('fashion_model')

image__path='images/2886.jpg'


image = load_image(image__path)
predictions = predict_fashion_item(model, label_binarizers, image)

for category, pred in predictions.items():
    print(f"{category}: {pred['label']} (confidence: {pred['confidence']:.2f})")

Model and components loaded from fashion_model
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
subCategory: Shoes (confidence: 1.00)
gender: Men (confidence: 0.62)
color: Red (confidence: 0.32)
season: Spring (confidence: 0.39)
usage: Casual (confidence: 1.00)


In [ ]:
from tensorflow.keras.models import Model

def create_embedding_model(trained_model):
    """
    Create a model that outputs embeddings from the trained model
    
    Args:
        trained_model: The trained CNN model
        
    Returns:
        Model that outputs embeddings
    """

    embedding_layer = None
    for layer in reversed(trained_model.layers):
      
        if not layer.name.endswith('output') and not 'dense' in layer.name.lower():
            embedding_layer = layer
            break
    
    if embedding_layer is None:
        raise ValueError("Could not find appropriate embedding layer")
    
   
    embedding_model = Model(
        inputs=trained_model.input,
        outputs=embedding_layer.output
    )
    
    return embedding_model

def extract_embeddings(model, images, batch_size=32):
    """
    Extract embeddings for a set of images
    
    Args:
        model: The trained model or embedding model
        images: Array of preprocessed images
        batch_size: Batch size for processing
        
    Returns:
        numpy array of embeddings
    """
   
    if any(layer.name.endswith('output') for layer in model.layers):
        embedding_model = create_embedding_model(model)
    else:
        embedding_model = model
    
   
    embeddings = embedding_model.predict(images, batch_size=batch_size)
    
    return embeddings

In [ ]:
# embed=create_embedding_model(model)
# embedding=extract_embeddings(embed,image_data)



In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_items(selected_embedding, all_embeddings, top_k=5):
    """
    Find the most similar items to a selected item using cosine similarity.
    
    Args:
        selected_embedding: The embedding vector of the selected item
        all_embeddings: Matrix of embeddings for all items
        top_k: Number of similar items to return
        
    Returns:
        List of indices of the most similar items
    """
    
    if len(selected_embedding.shape) == 1:
        selected_embedding = selected_embedding.reshape(1, -1)
    
    
    similarities = cosine_similarity(selected_embedding, all_embeddings)
    
    similar_indices = np.argsort(similarities[0])[::-1][1:top_k+1]
    
    return similar_indices.tolist()

def generate_outfit(selected_item_idx, clothing_item_embeddings, df, top_k=5):
    """
    Generate an outfit based on a selected clothing item

    Args:
        selected_item_idx: Index of the selected item in the dataset
        clothing_item_embeddings: Matrix of embeddings for all clothing items
        df: DataFrame containing item details
        top_k: Number of similar items to consider
    
    Returns:
        Dictionary containing the generated outfit
    """
    print("\nDebug: Starting generate_outfit function")
    print(f"Debug: Selected item index: {selected_item_idx}")
    
  
    selected_item_embedding = clothing_item_embeddings[selected_item_idx]
    selected_item = df.iloc[selected_item_idx]
    print(f"Debug: Selected item category: {selected_item['subCategory']}")
    
 
    similarities = cosine_similarity(
        selected_item_embedding.reshape(1, -1), 
        clothing_item_embeddings
    )
    similar_items_idx = np.argsort(similarities[0])[::-1][:top_k]
    print(f"Debug: Found {len(similar_items_idx)} similar items")
    
   
    category_groups = {
        'top': [],
        'bottom': [],
        'shoes': [],
        'accessory': []
    }
    
   
    category_mapping = {
        # Tops
        'Topwear': 'top',
        'Shirts': 'top',
        'T-shirts': 'top',
        'Tshirts': 'top',
        'Shirt': 'top',
        'T-shirt': 'top',
        'Top': 'top',
        'Jacket': 'top',
        'Sweater': 'top',
        'Sweatshirt': 'top',
        # Bottoms
        'Bottomwear': 'bottom',
        'Pants': 'bottom',
        'Jeans': 'bottom',
        'Shorts': 'bottom',
        'Trousers': 'bottom',
        'Skirt': 'bottom',
        # Shoes
        'Shoes': 'shoes',
        'Footwear': 'shoes',
        'Sneakers': 'shoes',
        'Boots': 'shoes',
        'Sandals': 'shoes',
        # Accessories
        'Accessory': 'accessory',
        'Accessories': 'accessory',
        'Watch': 'accessory',
        'Belt': 'accessory',
        'Bag': 'accessory',
        'Wallet': 'accessory'
    }
    
    print("\nDebug: Grouping items by category")
    
    selected_category = category_mapping.get(selected_item['subCategory'])
    if selected_category:
        category_groups[selected_category].append(selected_item_idx)
        print(f"Debug: Added selected item to {selected_category} category")
    
    # Group similar items by categories
    for idx in similar_items_idx:
        item = df.iloc[idx]
        category = category_mapping.get(item['subCategory'])
        if category and idx != selected_item_idx:  # Avoid duplicating selected item
            category_groups[category].append(idx)
            print(f"Debug: Added similar item {idx} to {category} category")
    
    # Print category group sizes
    for category, items in category_groups.items():
        print(f"Debug: {category} category has {len(items)} items")
    
    # Generate outfit
    outfit = {}
    
    # Helper function to add item to outfit
    def add_item_to_outfit(category, item_idx):
        outfit[category] = {
            'item': df.iloc[item_idx],
            'similarity_score': float(cosine_similarity(
                selected_item_embedding.reshape(1, -1),
                clothing_item_embeddings[item_idx].reshape(1, -1)
            )[0][0])
        }
        print(f"Debug: Added {category} item to outfit")
    
    # Add selected item first
    if selected_category:
        add_item_to_outfit(selected_category, selected_item_idx)
    
    # Add items for other categories
    for category, items in category_groups.items():
        if category != selected_category and items:  # Skip selected item's category
            add_item_to_outfit(category, items[0])  # Add first item from each category
    
    print(f"Debug: Final outfit has {len(outfit)} items")
    return outfit

def get_item_category(subcategory):
    """
    Helper function to map subcategories to main categories
    
    Args:
        subcategory: String representing the item's subcategory
        
    Returns:
        String representing the main category
    """
    category_mapping = {
        'top': ['Shirt', 'Jacket', 'Top', 'T-shirt', 'Sweater', 'Blouse'],
        'bottom': ['Pants', 'Shorts', 'Skirt', 'Jeans'],
        'shoes': ['Shoes', 'Sneakers', 'Boots', 'Sandals'],
        'accessory': ['Accessory', 'Bag', 'Belt', 'Hat', 'Jewelry']
    }
    
    for category, subcategories in category_mapping.items():
        if subcategory in subcategories:
            return category
    return None

In [ ]:

def display_outfit(outfit, save_path=None):
    """Display or save the recommended outfit images"""
    num_items = len(outfit)
    fig, axes = plt.subplots(1, num_items, figsize=(4*num_items, 4))
    
    if num_items == 1:
        axes = [axes]
    
    for ax, (category, item_data) in zip(axes, outfit.items()):
        image = item_data['image']
        
        if image.max() <= 1.0:
            image = (image * 255).astype(np.uint8)
        
        ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{category}\n{item_data['item']['subCategory']}\n"
                    f"Similarity: {item_data['similarity_score']:.2f}")
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

In [ ]:
def save_embeddings(embeddings, file_path):
    """Save embeddings to a file."""
    np.save(file_path, embeddings)
def load_embeddings(file_path):
    """Load embeddings from a file."""
    return np.load(file_path)

def compute_and_save_embeddings(embedding_model, image_data, save_path):
    """
        Compute embeddings for all images and save them to a file.
        
        Args:
            embedding_model: The embedding model
            image_data: NumPy array of processed images
            save_path: Path to save the embeddings
    """
    if os.path.exists(save_path):
        print(f"Loading precomputed embeddings from {save_path}...")
        embeddings = load_embeddings(save_path)
    else:
        print("Computing embeddings...")
        embeddings = embedding_model.predict(np.array(image_data), batch_size=32)
        save_embeddings(embeddings, save_path)
        print(f"Embeddings saved to {save_path}.")
        
    return embeddings


    
# 
    
   
    

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import matplotlib.pyplot as plt
import os

def process_input_image(image_path, classification_model, label_binarizers):
    """
    Process a single input image and extract its features
    
    Args:
        image_path: Path to the input image
        classification_model: Trained multi-label classification model
        label_binarizers: Dictionary of label binarizers
        
    Returns:
        dict: Extracted features with confidence scores
        array: Preprocessed image
    """

    image = load_image(image_path)
    if image is None:
        raise ValueError("Could not load input image")
    

    image_batch = np.expand_dims(image, axis=0)
    predictions = predict_fashion_item(classification_model, label_binarizers, image_batch)
    
    return predictions, image

def create_embedding_model(trained_model):
    """
    Create a model that outputs embeddings from the trained model
    """
  
    embedding_layer = None
    for layer in reversed(trained_model.layers):
        if not layer.name.endswith('output') and not 'dense' in layer.name.lower():
            embedding_layer = layer
            break
    
    if embedding_layer is None:
        raise ValueError("Could not find appropriate embedding layer")
    
 
    embedding_model = Model(
        inputs=trained_model.input,
        outputs=embedding_layer.output
    )
    
    return embedding_model

def filter_similar_items(similar_items_idx, df, input_features, min_confidence=0.7):
    """
    Filter similar items based on classification features
    
    Args:
        similar_items_idx: List of indices of similar items
        df: DataFrame containing item metadata
        input_features: Features extracted from input image
        min_confidence: Minimum confidence score to consider a feature match
    
    Returns:
        List of filtered item indices
    """
    filtered_idx = []
    
    # Get high-confidence features from input
    important_features = {
        k: v['label'] for k, v in input_features.items() 
        if float(v['confidence']) >= min_confidence
    }
    
    for idx in similar_items_idx:
        item = df.iloc[idx]
        # Check if item matches important features
        matches = True
        for feature, value in important_features.items():
            if feature in ['gender', 'usage', 'season']:
                # Convert to string and compare
                if str(item[feature]).lower() != str(value).lower():
                    matches = False
                    break
        if matches:
            filtered_idx.append(idx)
    
    # If no matches found, return original indices
    return filtered_idx if filtered_idx else list(similar_items_idx)

def generate_outfit(selected_item_idx, clothing_item_embeddings, df, top_k=10):
    """
    Generate an outfit ensuring items from different categories are selected
    
    Args:
        selected_item_idx: Index of the selected item
        clothing_item_embeddings: Matrix of embeddings for all items
        df: DataFrame containing item metadata
        top_k: Number of similar items to consider
    """
    print("\nDebug: Starting generate_outfit function")
    
    # Get selected item's embedding and features
    selected_embedding = clothing_item_embeddings[selected_item_idx]
    selected_item = df.iloc[selected_item_idx]
    
    # Calculate similarities with all items
    similarities = cosine_similarity(
        selected_embedding.reshape(1, -1), 
        clothing_item_embeddings
    )[0]
    
    # Get all items sorted by similarity
    all_items_idx = np.argsort(similarities)[::-1]
    
    # Initialize category groups
    category_groups = {
        'top': [],
        'bottom': [],
        'shoes': [],
        'accessory': []
    }
    
    # Map subcategories to main categories
    category_mapping = {
        'top': ['Topwear', 'Shirt', 'T-shirt', 'Top'],
        'bottom': ['Bottomwear', 'Pants', 'Jeans', 'Shorts', 'Skirt'],
        'shoes': ['Shoes', 'Footwear', 'Sneakers', 'Sandals', 'Flip Flops'],
        'accessory': ['Accessories', 'Watch', 'Belt', 'Wallet', 'Bag']
    }
    
    # Get selected item's features
    selected_gender = df.iloc[selected_item_idx]['gender']
    selected_usage = df.iloc[selected_item_idx]['usage']
    
    # Function to get main category
    def get_main_category(subcategory):
        for main_cat, sub_cats in category_mapping.items():
            if any(sub.lower() in subcategory.lower() for sub in sub_cats):
                return main_cat
        return None
    
    # First, add the selected item to its category
    selected_category = get_main_category(selected_item['subCategory'])
    if selected_category:
        category_groups[selected_category].append({
            'idx': selected_item_idx,
            'similarity': 1.0
        })
    
    print(f"Debug: Selected item category: {selected_category}")
    
    # Then find items for other categories
    items_per_category = 5  # Number of items to collect per category
    
    for idx in all_items_idx:
        item = df.iloc[idx]
        
        # Skip if it's the selected item
        if idx == selected_item_idx:
            continue
            
        # Only consider items matching gender and usage
        if item['gender'] != selected_gender or item['usage'] != selected_usage:
            continue
        
        # Get the main category for this item
        main_category = get_main_category(item['subCategory'])
        if main_category and len(category_groups[main_category]) < items_per_category:
            category_groups[main_category].append({
                'idx': idx,
                'similarity': similarities[idx]
            })
    
    # Print debug information
    for category, items in category_groups.items():
        print(f"Debug: {category} category has {len(items)} items")
    
    # Create final outfit by selecting top item from each category
    outfit = {}
    for category, items in category_groups.items():
        if items:  
           
            if category == selected_category:
                item_data = items[0]  
            else:
             
                item_data = max(items, key=lambda x: x['similarity'])
            
            outfit[category] = {
                'item': df.iloc[item_data['idx']],
                'similarity_score': item_data['similarity']
            }
            print(f"Debug: Added {category} item to outfit")
    
    print(f"Debug: Final outfit has {len(outfit)} items")
    return outfit

def generate_complete_outfit(image_path, classification_model, label_binarizers, 
                           embedding_model, image_data, df, top_k=10):
    """Generate a complete outfit based on a single input image"""
    print("\nStarting outfit generation process...")
    
 
    print("Processing input image...")
    input_features, processed_image = process_input_image(
        image_path, classification_model, label_binarizers
    )
    print("Input features extracted:", input_features)
    
  
    original_input_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    
   
    print("\nGenerating embeddings for input image...")
    input_embedding = embedding_model.predict(np.expand_dims(processed_image, axis=0))
    
    # print("Generating embeddings for all images...")
    # all_embeddings = embedding_model.predict(np.array(image_data), batch_size=32)
    embeddings_save_path = "precomputed_embeddings.npy"
    all_embeddings = compute_and_save_embeddings(embedding_model, image_data, embeddings_save_path)

   
    print("\nFinding similar items...")
    similarities = cosine_similarity(input_embedding, all_embeddings)
    similar_items_idx = np.argsort(similarities[0])[::-1][:top_k]
    print(f"Found {len(similar_items_idx)} similar items")
    
  
    print("\nFiltering similar items...")
    filtered_items_idx = filter_similar_items(
        similar_items_idx, df, input_features
    )
    print(f"Filtered to {len(filtered_items_idx)} items")
    
    if not filtered_items_idx:
        print("Warning: No items found after filtering!")
        return {}, input_features, original_input_image
    
   
    print("\nGenerating outfit...")
    outfit = generate_outfit(
        selected_item_idx=filtered_items_idx[0],
        clothing_item_embeddings=all_embeddings,
        df=df,
        top_k=top_k
    )
    
    print(f"\nOutfit generated with {len(outfit)} items")
    
    
    print("Adding images to outfit...")
    for category, item_data in outfit.items():
        try:
            idx = df.index.get_loc(item_data['item'].name)
            outfit[category]['image'] = image_data[idx]
            print(f"Added image for {category}")
        except Exception as e:
            print(f"Error adding image for {category}: {str(e)}")
    
    return outfit, input_features, original_input_image

# def main(image_data, df):
#     print("\nInitializing main function...")
#     print(f"Image data shape: {image_data.shape}")
#     print(f"DataFrame shape: {df.shape}")
    
#     matcher, _, _, _ = setup_matcher(
#     classification_model_path='fashion_model',
#     image_dir="images1"
#     )
  
#     print("\nLoading models...")
#     classification_model, label_binarizers, num_classes_dict = load_fashion_model('fashion_model')
#     embedding_model = create_embedding_model(classification_model)
    
#     input_image_path = "images/abc.jpg"
#     print(f"\nProcessing input image: {input_image_path}")
    
    
#     outfit, input_features, input_image = generate_complete_outfit(
#         image_path=input_image_path,
#         classification_model=classification_model,
#         label_binarizers=label_binarizers,
#         embedding_model=embedding_model,
#         image_data=image_data,
#         df=df
#     )
    
#     if not outfit:
#         print("\nNo outfit was generated!")
#         return
    

#     print("\nInput Item Features:")
#     for feature, data in input_features.items():
#         print(f"{feature}: {data['label']} (confidence: {data['confidence']:.2f})")
    

#     print("\nDisplaying outfit...")
#     display_outfit(outfit)
    

#     print("\nRecommended Outfit Details:")
#     for category, item_data in outfit.items():
#         print(f"\n{category.upper()}:")
#         print(f"Item: {item_data['item']['subCategory']}")
#         print(f"Color: {item_data['item'].get('color', 'N/A')}")
#         print(f"Similarity Score: {item_data['similarity_score']:.2f}")





# if __name__ == "__main__":
    
#     main(image_data, df)

In [ ]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
from datetime import datetime

class ImageMatcher:
    def __init__(self, classification_model, embedding_model, label_binarizers, base_image_dir, cache_dir='./features_cache'):
        """
        Initialize the comprehensive image matching system
        
        Args:
            classification_model: Trained classification model
            embedding_model: Model for feature extraction
            label_binarizers: Dictionary of label binarizers for each category
            base_image_dir: Path to the main images directory
            cache_dir: Directory to store processed image features
        """
        self.base_image_dir = Path(base_image_dir)
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        
        self.classification_model = classification_model
        self.embedding_model = embedding_model
        self.label_binarizers = label_binarizers
        
        # Cache paths
        self.cache_metadata_path = self.cache_dir / 'cache_metadata.json'
        self.category_index_path = self.cache_dir / 'category_index.json'
        self.metadata_path = self.cache_dir / 'generated_metadata.json'
        self.category_stats_path = self.cache_dir / 'category_statistics.json'
        
        # Initialize data structures
        self.image_features = {}
        self.category_index = {}
        self.metadata = {}
        self.category_stats = {}
        self.loaded_categories = set()
        
        # Load or initialize metadata
        self.load_cache_metadata()
        self.generate_or_load_metadata()
        
    def load_cache_metadata(self):
        """Load or initialize cache metadata"""
        if self.cache_metadata_path.exists():
            with open(self.cache_metadata_path, 'r') as f:
                self.cache_metadata = json.load(f)
        else:
            self.cache_metadata = {
                'last_update': None,
                'processed_images': {}
            }
    
    def generate_or_load_metadata(self):
        """Generate or load metadata and category statistics"""
        if self.metadata_path.exists():
            print("Loading existing metadata...")
            with open(self.metadata_path, 'r') as f:
                self.metadata = json.load(f)
            with open(self.category_stats_path, 'r') as f:
                self.category_stats = json.load(f)
            print(f"Loaded metadata for {len(self.metadata)} images")
            self._print_category_statistics()
        else:
            print("Generating new metadata for all images...")
            self._generate_metadata()
    
    def _generate_metadata(self):
        """Process all images to generate metadata and category statistics"""
        batch_size = 32
        all_images = list(Path(self.base_image_dir).rglob('*.[jJ][pP][gG]'))
        total_images = len(all_images)
        
        self.category_stats = {
            'total_images': total_images,
            'categories': {},
            'category_combinations': {}
        }
        
        print(f"Processing {total_images} images...")
        
        for i in tqdm(range(0, total_images, batch_size), desc="Generating metadata"):
            batch_paths = all_images[i:i + batch_size]
            batch_arrays = []
            valid_paths = []
            
            for path in batch_paths:
                try:
                    img_array = self.preprocess_image(path)
                    if img_array is not None:
                        batch_arrays.append(img_array[0])
                        valid_paths.append(path)
                except Exception as e:
                    print(f"Error preprocessing image {path}: {e}")
                    continue
            
            if batch_arrays:
                try:
                    batch_array = np.array(batch_arrays)
                    features = self.extract_features(batch_array)
                    
                    for idx, path in enumerate(valid_paths):
                        try:
                            article_id = path.stem
                            
                            # Add debugging output
                            print(f"Processing {article_id}")
                            print("Classifications shape:", 
                                  np.array(features['classifications'][idx]).shape)
                            
                            categories = self._process_predictions(features['classifications'][idx])
                            
                            # Store metadata
                            self.metadata[article_id] = {
                                'path': str(path),
                                'categories': categories['categories'],
                                'confidence_scores': categories['scores'],
                                'primary_category': categories['primary_category']
                            }
                            
                            # Update category statistics
                            self._update_category_stats(categories['categories'])
                            
                            # Cache features
                            feature_path = self.cache_dir / f"{article_id}_features.npz"
                            np.savez(
                                feature_path,
                                classifications=features['classifications'][idx],
                                embeddings=features['embeddings'][idx]
                            )
                        except Exception as e:
                            print(f"Error processing image {path}: {e}")
                            continue
                except Exception as e:
                    print(f"Error processing batch: {e}")
                    continue
        
        self._finalize_statistics()
        self._save_metadata()
        
    def preprocess_image(self, image_path):
        """Preprocess image for model input"""
        try:
            img = Image.open(image_path)
            img = img.convert('RGB')
            img = img.resize((180, 180))
            img_array = img_to_array(img)
            img_array = img_array / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            return img_array
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            return None
    
    def extract_features(self, image_array):
        """Extract features using both models"""
        # Get raw predictions
        classifications = self.classification_model.predict(image_array)
        embeddings = self.embedding_model.predict(image_array)
        
        # Ensure consistent format for classifications
        if isinstance(classifications, dict):
        # If classifications is a dictionary, take the first key's predictions
            pred_key = list(classifications.keys())[0]
            classifications = classifications[pred_key]
    
        classifications = np.array(classifications)
    
        if len(image_array.shape) == 3:  # Single image
            return {
                'classifications': [classifications],
                'embeddings': [embeddings] if isinstance(embeddings, np.ndarray) else embeddings
            }
        else:  # Batch of images
            return {
                'classifications': [classifications[i] if classifications.ndim > 1 else classifications 
                              for i in range(len(image_array))],
                'embeddings': embeddings
            }
    
    def _process_predictions(self, predictions):
        """Process model predictions into categories with confidence scores"""
        threshold = 0.3
        categories = []
        scores = {}
        
        # Add debugging output
        print("Predictions type:", type(predictions))
        print("Predictions shape:", np.array(predictions).shape)
        print("Predictions content:", predictions)
        
        # Convert predictions to numpy array if needed
        if not isinstance(predictions, np.ndarray):
            predictions = np.array(predictions)
        
        # Ensure predictions is 1-dimensional
        predictions = predictions.flatten()
        
        category_names = ['mens', 'womens', 'children', 'shirts', 'pants', 'shoes', 'accessories']
        for idx, score in enumerate(predictions[:len(category_names)]):
            if score > threshold:
                cat = category_names[idx]
                categories.append(cat)
                scores[cat] = float(score)
        
        primary_category = max(scores.items(), key=lambda x: x[1])[0] if scores else None
    
        return {
            'categories': categories,
            'scores': scores,
            'primary_category': primary_category
        }
    def _update_category_stats(self, categories):
        """Update category statistics"""
        for cat in categories:
            if cat not in self.category_stats['categories']:
                self.category_stats['categories'][cat] = 0
            self.category_stats['categories'][cat] += 1
        
        combo = '+'.join(sorted(categories))
        if combo not in self.category_stats['category_combinations']:
            self.category_stats['category_combinations'][combo] = 0
        self.category_stats['category_combinations'][combo] += 1
    
    def _finalize_statistics(self):
        """Finalize category statistics with percentages"""
        total = self.category_stats['total_images']
        for cat, count in self.category_stats['categories'].items():
            self.category_stats['categories'][cat] = {
                'count': count,
                'percentage': (count / total) * 100
            }
    
    def _save_metadata(self):
        """Save all metadata to files"""
        with open(self.metadata_path, 'w') as f:
            json.dump(self.metadata, f)
        with open(self.category_stats_path, 'w') as f:
            json.dump(self.category_stats, f)
        with open(self.cache_metadata_path, 'w') as f:
            json.dump(self.cache_metadata, f)
    
    def find_similar_images(self, query_features, categories=None, top_k=5):
        """
        Find similar images with optional category filtering
        
        Args:
            query_features: Features of the query image
            categories: Optional list of categories to search within
            top_k: Number of matches to return
        """
        if categories:
            self.load_category_features(categories)
        
        similarities = {}
        query_embedding = query_features['embeddings'].flatten()
        query_classifications = query_features['classifications']
        
        for article_id, features in self.image_features.items():
            if categories and not any(cat in self.metadata[article_id]['categories'] 
                                    for cat in categories):
                continue
            
            db_embedding = features['embeddings'].flatten()
            embedding_similarity = float(
                np.dot(query_embedding, db_embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(db_embedding)
                )
            )
            
            classification_similarity = self._calculate_classification_similarity(
                query_classifications, features['classifications']
            )
            
            similarity = (0.6 * embedding_similarity + 0.4 * classification_similarity)
            similarities[article_id] = {
                'total': float(similarity),
                'embedding': float(embedding_similarity),
                'classification': float(classification_similarity)
            }
        
        return self._get_top_matches(similarities, top_k)
    
    def _calculate_classification_similarity(self, query_pred, db_pred):
        """Calculate similarity between classification predictions"""
        if isinstance(query_pred, dict):
            query_pred = next(iter(query_pred.values()))
        if isinstance(db_pred, dict):
            db_pred = next(iter(db_pred.values()))
            
        query_pred = np.array(query_pred).flatten()
        db_pred = np.array(db_pred).flatten()
        
        if np.any(query_pred) and np.any(db_pred):
            return float(
                np.dot(query_pred, db_pred) / (
                    np.linalg.norm(query_pred) * np.linalg.norm(db_pred)
                )
            )
        return 0.0
    
    def _get_top_matches(self, similarities, top_k):
        """Get top k matches with detailed information"""
        sorted_items = sorted(
            similarities.items(),
            key=lambda x: x[1]['total'],
            reverse=True
        )[:top_k]
        
        results = []
        for article_id, scores in sorted_items:
            result = {
                'article_id': article_id,
                'similarity_score': scores['total'],
                'embedding_sim': scores['embedding'],
                'classification_sim': scores['classification'],
                'image_path': str(self.metadata[article_id]['path']),
                'categories': self.metadata[article_id]['categories'],
                'confidence_scores': self.metadata[article_id]['confidence_scores']
            }
            results.append(result)
        
        return results
    
    def visualize_matches(self, query_image_path, matches, title="Matching Results"):
        """Visualize matching results with detailed information"""
        n_matches = len(matches)
        fig = plt.figure(figsize=(15, 8))
        
        # Query image
        plt.subplot(2, n_matches + 1, 1)
        query_img = Image.open(query_image_path)
        plt.imshow(query_img)
        plt.title("Query Image")
        plt.axis('off')
        
        # Matches
        for idx, match in enumerate(matches):
            # Image
            plt.subplot(2, n_matches + 1, idx + 2)
            try:
                img = Image.open(match['image_path'])
                plt.imshow(img)
                plt.title(f"Match {idx + 1}\nScore: {match['similarity_score']:.2f}")
            except Exception as e:
                plt.text(0.5, 0.5, f"Error loading image:\n{str(e)}", 
                        ha='center', va='center')
            plt.axis('off')
            
            # Details
            plt.subplot(2, n_matches + 1, n_matches + idx + 3)
            details = (f"Embedding: {match['embedding_sim']:.2f}\n"
                      f"Classification: {match['classification_sim']:.2f}\n"
                      f"Categories: {', '.join(match['categories'])}")
            plt.text(0.5, 0.5, details, ha='center', va='center')
            plt.axis('off')
        
        plt.suptitle(title)
        plt.tight_layout()
        return fig

    def get_recommendations(self, input_image_path, categories=None, top_k=5):
        """
        Get recommendations for an input image
        
        Args:
            input_image_path: Path to the query image
            categories: Optional list of categories to search within
            top_k: Number of recommendations to return
        """
        input_array = self.preprocess_image(input_image_path)
        if input_array is None:
            raise ValueError("Could not process input image")
            
        input_features = self.extract_features(input_array)
        matches = self.find_similar_images(input_features, categories, top_k)
        
        return matches

In [ ]:
classification_model, label_binarizers, num_classes_dict = load_fashion_model('fashion_model')
embedding_model = create_embedding_model(classification_model)

matcher = ImageMatcher(
    classification_model=classification_model,
    embedding_model=embedding_model,
    label_binarizers=label_binarizers,
    base_image_dir='images1'
)

recommendations = matcher.get_recommendations(
    input_image_path='images/28865.jpg',
    # categories=['shirts', 'pants'],  # optional
    top_k=5
)

matcher.visualize_matches(
    query_image_path='images/28865.jpg',
    matches=recommendations
)

In [ ]:
#1

In [ ]:
#2

In [ ]:
#3

In [ ]:
#4

In [ ]:
# import os
# import numpy as np
# from pathlib import Path
# from PIL import Image
# import tensorflow as tf
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.preprocessing.image import img_to_array
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import json
# from datetime import datetime

# class EnhancedImageMatcher:
#     def __init__(self, classification_model, embedding_model, label_binarizers, base_image_dir, cache_dir='./features_cache'):
#         """
#         Initialize the matching system with complete model setup
        
#         Args:
#             classification_model: Your trained classification model
#             embedding_model: Your embedding model for feature extraction
#             label_binarizers: Dictionary of label binarizers for each category
#             base_image_dir: Path to the main images directory
#             cache_dir: Directory to store processed image features
#         """
#         self.base_image_dir = Path(base_image_dir)
#         self.cache_dir = Path(cache_dir)
#         self.cache_dir.mkdir(exist_ok=True)
        
#         self.classification_model = classification_model
#         self.embedding_model = embedding_model
#         self.label_binarizers = label_binarizers
        
#         self.cache_metadata_path = self.cache_dir / 'cache_metadata.json'
#         self.load_cache_metadata()
        
#         self.map_available_images()

#     def extract_features(self, image_array):
#         """Extract features using both classification and embedding models"""
#         classifications = self.classification_model.predict(image_array)
#         if not isinstance(classifications, dict):
#             classifications = {'prediction': classifications}
            
#         embeddings = self.embedding_model.predict(image_array)
        
#         return {
#             'classifications': classifications,
#             'embeddings': embeddings
#         }
    
#     def load_cache_metadata(self):
#         """Load or initialize cache metadata"""
#         if self.cache_metadata_path.exists():
#             with open(self.cache_metadata_path, 'r') as f:
#                 self.cache_metadata = json.load(f)
#         else:
#             self.cache_metadata = {
#                 'last_update': None,
#                 'processed_images': {}
#             }
    
#     def save_cache_metadata(self):
#         """Save cache metadata"""
#         with open(self.cache_metadata_path, 'w') as f:
#             json.dump(self.cache_metadata, f)
    
#     def map_available_images(self):
#         """Find all available images in nested directory structure"""
#         print("Mapping available images...")
#         self.available_images = {}
        
#         # Recursively walk through all subdirectories
#         for root, _, files in tqdm(os.walk(self.base_image_dir)):
#             for file in files:
#                 if file.lower().endswith(('.jpg', '.jpeg', '.png')):
#                     article_id = Path(file).stem
#                     full_path = Path(root) / file
#                     self.available_images[article_id] = full_path
        
#         print(f"Found {len(self.available_images)} images")
    
#     def preprocess_image(self, image_path):
#         """Preprocess image exactly as done in the original CNN model"""
#         try:
#             img = Image.open(image_path)
#             img = img.convert('RGB')  
#             img = img.resize((180, 180))  
#             img_array = img_to_array(img)
#             img_array = img_array / 255.0  
#             img_array = np.expand_dims(img_array, axis=0)
#             return img_array
#         except Exception as e:
#             print(f"Error processing image {image_path}: {e}")
#             return None
    
#     def process_database_images(self, batch_size=32, force_reprocess=False):
#         """
#         Process all available images and cache their features
        
#         Args:
#             batch_size: Number of images to process at once
#             force_reprocess: If True, reprocess all images even if cached
#         """
#         print("Loading/extracting features from database images...")
#         self.image_features = {}
        
#         cached_count = 0
#         if not force_reprocess:
#             for article_id in tqdm(self.available_images.keys(), desc="Loading cached features"):
#                 feature_path = self.cache_dir / f"{article_id}_features.npz"
#                 if feature_path.exists():
#                     try:
#                         with np.load(feature_path, allow_pickle=True) as data:
#                             self.image_features[article_id] = {
#                                 'classifications': data['classifications'].item(),
#                                 'embeddings': data['embeddings']
#                             }
#                             cached_count += 1
#                     except Exception as e:
#                         print(f"Error loading cached features for {article_id}: {e}")
        
#         print(f"Loaded {cached_count} cached features")
        
#         remaining_ids = [aid for aid in self.available_images.keys() 
#                         if aid not in self.image_features or force_reprocess]
        
#         if remaining_ids:
#             print(f"Processing {len(remaining_ids)} new images...")
#             for i in tqdm(range(0, len(remaining_ids), batch_size), desc="Processing new images"):
#                 batch_ids = remaining_ids[i:i + batch_size]
#                 batch_images = []
#                 valid_ids = []
                
#                 for article_id in batch_ids:
#                     img_array = self.preprocess_image(self.available_images[article_id])
#                     if img_array is not None:
#                         batch_images.append(img_array[0])
#                         valid_ids.append(article_id)
                
#                 if batch_images:
#                     batch_array = np.array(batch_images)
#                     features = self.extract_features(batch_array)
                    
#                     for idx, article_id in enumerate(valid_ids):
#                         image_features = {
#                             'classifications': {
#                                 name: pred[idx] for name, pred in features['classifications'].items()
#                             },
#                             'embeddings': features['embeddings'][idx]
#                         }
                        
#                         self.image_features[article_id] = image_features
                        
#                         # Save to cache immediately after processing each batch
#                         np.savez(
#                             self.cache_dir / f"{article_id}_features.npz",
#                             classifications=image_features['classifications'],
#                             embeddings=image_features['embeddings']
#                         )
                        
#                         self.cache_metadata['processed_images'][article_id] = {
#                             'last_processed': datetime.now().isoformat(),
#                             'feature_shapes': {
#                                 'classifications': {
#                                     name: preds.shape if hasattr(preds, 'shape') else None
#                                     for name, preds in image_features['classifications'].items()
#                                 },
#                                 'embeddings': image_features['embeddings'].shape
#                             }
#                         }
                        
#                 # Save metadata after each batch
#                 self.save_cache_metadata()
        
#         self.cache_metadata['last_update'] = datetime.now().isoformat()
#         self.save_cache_metadata()
        
#         print(f"Total processed features: {len(self.image_features)}")
#     def find_similar_images(self, query_features, top_k=5):
#         """
#         Find similar images based on both embeddings and classifications.
#         Fixed to properly handle array operations and similarity calculations.
        
#         Args:
#             query_features: Dictionary containing 'classifications' and 'embeddings'
#             top_k: Number of top matches to return
#         Returns:
#             List of dictionaries containing match information
#         """
#         similarities = {}
#         query_embedding = query_features['embeddings'].flatten()
#         query_classifications = query_features['classifications']
        
#         for article_id, features in self.image_features.items():
#             # Calculate embedding similarity using cosine similarity
#             db_embedding = features['embeddings'].flatten()
#             embedding_similarity = float(
#                 np.dot(query_embedding, db_embedding) / (
#                     np.linalg.norm(query_embedding) * np.linalg.norm(db_embedding)
#                 )
#             )
            
#             # Calculate classification similarity
#             classification_similarity = 0.0
#             total_classes = 0
#             for name in query_classifications.keys():
#                 if name in features['classifications']:
#                     # Handle multi-class predictions properly
#                     query_pred = query_classifications[name]
#                     db_pred = features['classifications'][name]
                    
#                     # Ensure we're working with flattened arrays
#                     query_pred = np.array(query_pred).flatten()
#                     db_pred = np.array(db_pred).flatten()
                    
#                     # Calculate cosine similarity for this classification
#                     if np.any(query_pred) and np.any(db_pred):  # Avoid zero vectors
#                         sim = float(
#                             np.dot(query_pred, db_pred) / (
#                                 np.linalg.norm(query_pred) * np.linalg.norm(db_pred)
#                             )
#                         )
#                         classification_similarity += sim
#                         total_classes += 1
            
#             # Normalize classification similarity
#             if total_classes > 0:
#                 classification_similarity /= total_classes
            
#             # Combine similarities with weighted average
#             # Adjust these weights based on which feature type is more important
#             similarity = (0.6 * embedding_similarity + 0.4 * classification_similarity)
#             similarities[article_id] = float(similarity)
    
#         # Get top k matches
#         similar_items = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_k]
        
#         return [
#             {
#                 'article_id': article_id,
#                 'similarity_score': float(similarity),
#                 'image_path': str(self.available_images[article_id]),
#                 'features': self.image_features[article_id],
#                 'embedding_sim': float(embedding_similarity),  # Added for debugging
#                 'classification_sim': float(classification_similarity)  # Added for debugging
#             }
#             for article_id, similarity in similar_items
#         ]
    
#     def visualize_matches(self, original_features, matches, title="Matching Results"):
#         """
#         Enhanced visualization method with more detailed similarity information
#         """
#         n_matches = len(matches)
#         fig = plt.figure(figsize=(15, 6))
        
#         # Plot original features using embeddings
#         plt.subplot(1, n_matches + 1, 1)
#         embedding_array = original_features['embeddings'].flatten()
#         square_size = int(np.sqrt(len(embedding_array)))
#         display_array = embedding_array[:square_size * square_size].reshape((square_size, -1))
#         plt.imshow(display_array, cmap='viridis')
#         plt.title("Query\nImage")
#         plt.axis('off')
        
#         # Plot matching images
#         for idx, match in enumerate(matches):
#             plt.subplot(1, n_matches + 1, idx + 2)
#             try:
#                 img = Image.open(match['image_path'])
#                 plt.imshow(img)
#                 # Show both embedding and classification similarities
#                 plt.title(f"Match {idx + 1}\nTotal: {match['similarity_score']:.2f}\n"
#                          f"Emb: {match.get('embedding_sim', 0):.2f}\n"
#                          f"Class: {match.get('classification_sim', 0):.2f}")
#             except Exception as e:
#                 plt.text(0.5, 0.5, f"Error loading image:\n{str(e)}", 
#                         ha='center', va='center')
#             plt.axis('off')
        
#         plt.suptitle(title)
#         plt.tight_layout()
#         return fig
        
#         def match_outfit_recommendations(self, recommendations, top_k=5, visualize=True):
#             """
#             Match each recommended feature vector to similar images
            
#             Args:
#                 recommendations: Dictionary of category-feature pairs from your CNN model
#                 top_k: Number of matches to return per item
#                 visualize: If True, create visualization plots
            
#             Returns:
#                 Dictionary mapping categories to lists of similar images and optional plots
#             """
#             matched_outfit = {}
#             visualizations = {}
            
#             for category, features in recommendations.items():
#                 similar_images = self.find_similar_images(features, top_k=top_k)
#                 matched_outfit[category] = similar_images
                
#                 if visualize:
#                     fig = self.visualize_matches(
#                         features,
#                         similar_images,
#                         title=f"Matches for {category}"
#                     )
#                     visualizations[category] = fig
            
#             return matched_outfit, visualizations if visualize else matched_outfit



In [ ]:
# def setup_matcher(classification_model_path, image_dir):
#     """Helper function to create and initialize the matching system"""
#     print("Loading models and components...")
#     classification_model, label_binarizers, num_classes_dict = load_fashion_model(classification_model_path)
#     embedding_model = create_embedding_model(classification_model)
    
#     print("Initializing matcher...")
#     matcher = EnhancedImageMatcher(
#         classification_model=classification_model,
#         embedding_model=embedding_model,
#         label_binarizers=label_binarizers,
#         base_image_dir=image_dir
#     )
    
#     print("Processing database images...")
#     matcher.process_database_images()
    
#     return matcher, classification_model, embedding_model, label_binarizers

# matcher, classification_model, embedding_model, label_binarizers = setup_matcher(
#     classification_model_path='fashion_model',
#     image_dir="images1"
# )

# input_image_path = "images/28865.jpg"
# input_array = matcher.preprocess_image(input_image_path)
# input_features = matcher.extract_features(input_array)

# matched_items = matcher.find_similar_images(
#     input_features,
#     top_k=5
# )

# matcher.visualize_matches(input_features, matched_items, "Similar Items")
# plt.show()

In [ ]:
# matcher, classification_model, embedding_model, label_binarizers = setup_matcher(
#     classification_model_path='fashion_model',
#     image_dir="images1"
# )

# input_image_path = "images/28863.jpg"
# input_array = matcher.preprocess_image(input_image_path)
# input_features = matcher.extract_features(input_array)

# matched_items = matcher.find_similar_images(
#     input_features,
#     top_k=5
# )

# matcher.visualize_matches(input_features, matched_items, "Similar Items")
# plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model

# try:
#     classification_model = load_model('classification_model')  
# except Exception as e:
#     print(f"Error loading model: {e}")
#     raise

# matcher = setup_matcher(
#     cnn_model=classification_model,
#     image_dir="images1"
# )

# recommendations = {
#     'tops': top_features,      
#     'bottoms': bottom_features,  
#     'shoes': shoe_features      
# }

# # Get matches and visualizations
# matched_outfit, visualizations = matcher.match_outfit_recommendations(
#     recommendations,
#     top_k=5,
#     visualize=True
# )

In [ ]:

# class ClusteredImageMatcher:
#     def __init__(self, classification_model, embedding_model, label_binarizers, base_image_dir, cache_dir='./features_cache'):
#         self.base_image_dir = Path(base_image_dir)
#         self.cache_dir = Path(cache_dir)
#         self.cache_dir.mkdir(exist_ok=True)
#         self.classification_model = classification_model
#         self.embedding_model = embedding_model
#         self.label_binarizers = label_binarizers
#         self.cache_metadata_path = self.cache_dir / 'cache_metadata.json'
#         self.category_index_path = self.cache_dir / 'category_index.json'
#         self.image_features = {}
#         self.category_index = {}
#         self.loaded_categories = set()
#         self.load_cache_metadata()
#         self.create_or_load_category_index()
        
#     def create_or_load_category_index(self):
#         """Create or load category index mapping images to their categories"""
#         if self.category_index_path.exists():
#             with open(self.category_index_path, 'r') as f:
#                 self.category_index = json.load(f)
#             print(f"Loaded category index with {len(self.category_index)} entries")
#         else:
#             print("Creating new category index...")
#             self._create_category_index()
    
#     def _create_category_index(self):
#         """Create category index by processing all images once"""
#         self.category_index = {}
#         batch_size = 32
#         all_images = list(Path(self.base_image_dir).rglob('*.[jJ][pP][gG]'))
        
#         for i in tqdm(range(0, len(all_images), batch_size), desc="Creating category index"):
#             batch_paths = all_images[i:i + batch_size]
#             batch_arrays = []
#             valid_paths = []
            
#             for path in batch_paths:
#                 img_array = self.preprocess_image(path)
#                 if img_array is not None:
#                     batch_arrays.append(img_array[0])
#                     valid_paths.append(path)
            
#             if batch_arrays:
#                 batch_array = np.array(batch_arrays)
#                 features = self.classification_model.predict(batch_array)
                
#                 for idx, path in enumerate(valid_paths):
#                     article_id = path.stem
#                     categories = self._get_categories_from_predictions(features[idx])
#                     self.category_index[article_id] = {
#                         'path': str(path),
#                         'categories': categories
#                     }
        
#         with open(self.category_index_path, 'w') as f:
#             json.dump(self.category_index, f)
        
#         print(f"Created category index with {len(self.category_index)} entries")
    
#     def _get_categories_from_predictions(self, predictions):
#         """Extract categories from model predictions"""
#         categories = []
#         threshold = 0.5  
#         if isinstance(predictions, np.ndarray):
#             categories = [
#                 cat for idx, cat in enumerate(['mens', 'womens', 'children', 
#                 'shirts', 'pants', 'shoes', 'accessories'])
#                 if predictions[idx] > threshold
#             ]
        
#         return categories
    
#     def _get_relevant_categories(self, query_features):
#         """Determine relevant categories for a query image"""
#         query_categories = self._get_categories_from_predictions(
#             query_features['classifications']
#         )
        
#         category_relations = {
#             'mens_shirts': ['mens_pants', 'mens_shoes', 'mens_accessories'],
#             'mens_pants': ['mens_shirts', 'mens_shoes', 'mens_accessories'],
#             'womens_shirts': ['womens_pants', 'womens_shoes', 'womens_accessories'],
#         }
        
#         relevant_categories = set()
#         for cat in query_categories:
#             relevant_categories.add(cat)
#             if cat in category_relations:
#                 relevant_categories.update(category_relations[cat])
        
#         return list(relevant_categories)
    
#     def load_category_features(self, categories):
#         """Load features only for specified categories"""
#         new_categories = set(categories) - self.loaded_categories
        
#         if not new_categories:
#             return  
        
#         print(f"Loading features for categories: {new_categories}")
        
#         relevant_ids = [
#             article_id for article_id, info in self.category_index.items()
#             if any(cat in info['categories'] for cat in new_categories)
#         ]
        
#         for article_id in tqdm(relevant_ids, desc="Loading category features"):
#             feature_path = self.cache_dir / f"{article_id}_features.npz"
#             if feature_path.exists():
#                 with np.load(feature_path, allow_pickle=True) as data:
#                     self.image_features[article_id] = {
#                         'classifications': data['classifications'].item(),
#                         'embeddings': data['embeddings']
#                     }
        
#         self.loaded_categories.update(new_categories)
#         print(f"Loaded {len(relevant_ids)} new images")
    
#     def find_similar_images(self, query_features, top_k=5):
#         """Find similar images, loading only relevant categories"""
#         relevant_categories = self._get_relevant_categories(query_features)
#         self.load_category_features(relevant_categories)
        
#         similarities = {}
#         query_embedding = query_features['embeddings'].flatten()
#         query_classifications = query_features['classifications']
        
#         similarity_components = {}
        
#         for article_id, features in self.image_features.items():
#             db_embedding = features['embeddings'].flatten()
#             embedding_similarity = float(
#                 np.dot(query_embedding, db_embedding) / (
#                     np.linalg.norm(query_embedding) * np.linalg.norm(db_embedding)
#                 )
#             )
            
#             classification_similarity = 0.0
#             total_classes = 0
#             for name in query_classifications.keys():
#                 if name in features['classifications']:
#                     query_pred = query_classifications[name]
#                     db_pred = features['classifications'][name]
                    
#                     query_pred = np.array(query_pred).flatten()
#                     db_pred = np.array(db_pred).flatten()
                    
#                     if np.any(query_pred) and np.any(db_pred):
#                         sim = float(
#                             np.dot(query_pred, db_pred) / (
#                                 np.linalg.norm(query_pred) * np.linalg.norm(db_pred)
#                             )
#                         )
#                         classification_similarity += sim
#                         total_classes += 1
            
#             if total_classes > 0:
#                 classification_similarity /= total_classes
            
#             similarity = (0.6 * embedding_similarity + 0.4 * classification_similarity)
#             similarities[article_id] = float(similarity)
            
#             similarity_components[article_id] = {
#                 'embedding_sim': float(embedding_similarity),
#                 'classification_sim': float(classification_similarity)
#             }
        
#         similar_items = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_k]
        
#         results = []
#         for article_id, similarity in similar_items:
#             result = {
#                 'article_id': article_id,
#                 'similarity_score': float(similarity),
#                 'image_path': str(self.category_index[article_id]['path']),
#                 'features': self.image_features[article_id],
#                 'embedding_sim': similarity_components[article_id]['embedding_sim'],
#                 'classification_sim': similarity_components[article_id]['classification_sim'],
#                 'categories': self.category_index[article_id]['categories']
#             }
#             results.append(result)
        
#         return results

#     def get_recommendations(self, input_image_path, top_k=5):
#         """
#         Convenience method for getting recommendations
        
#         Args:
#             input_image_path: Path to the query image
#             top_k: Number of recommendations to return
#         Returns:
#             List of similar items with detailed similarity information
#         """
#         input_array = self.preprocess_image(input_image_path)
#         if input_array is None:
#             raise ValueError("Could not process input image")
            
#         input_features = self.extract_features(input_array)
        
#         matches = self.find_similar_images(input_features, top_k=top_k)
        
#         return matches

#     def visualize_recommendations(self, query_image_path, recommendations, title="Recommendation Results"):
#         """
#         Visualize recommendations with detailed similarity information
        
#         Args:
#             query_image_path: Path to the query image
#             recommendations: List of recommendation results
#             title: Title for the visualization
#         """
#         n_matches = len(recommendations)
#         fig = plt.figure(figsize=(15, 8))
        
#         plt.subplot(2, n_matches + 1, 1)
#         query_img = Image.open(query_image_path)
#         plt.imshow(query_img)
#         plt.title("Query Image")
#         plt.axis('off')
        
#         plt.subplot(2, n_matches + 1, n_matches + 2)
#         plt.text(0.5, 0.5, "Query Categories:\n" + 
#                 "\n".join(self._get_relevant_categories({'classifications': None})),
#                 ha='center', va='center')
#         plt.axis('off')
        
#         for idx, match in enumerate(recommendations):
#             plt.subplot(2, n_matches + 1, idx + 2)
#             try:
#                 img = Image.open(match['image_path'])
#                 plt.imshow(img)
#                 plt.title(f"Match {idx + 1}\nScore: {match['similarity_score']:.2f}")
#             except Exception as e:
#                 plt.text(0.5, 0.5, f"Error loading image:\n{str(e)}", 
#                         ha='center', va='center')
#             plt.axis('off')
            
#             # Details
#             plt.subplot(2, n_matches + 1, n_matches + idx + 3)
#             details = (f"Emb: {match['embedding_sim']:.2f}\n"
#                       f"Class: {match['classification_sim']:.2f}\n"
#                       f"Categories: {', '.join(match['categories'])}")
#             plt.text(0.5, 0.5, details, ha='center', va='center')
#             plt.axis('off')
        
#         plt.suptitle(title)
#         plt.tight_layout()
#         return fig
        


In [ ]:
# class MetadataGeneratorMatcher:
#     def __init__(self, classification_model, embedding_model, label_binarizers, base_image_dir, cache_dir='./features_cache'):
#         self.base_image_dir = Path(base_image_dir)
#         self.cache_dir = Path(cache_dir)
#         self.cache_dir.mkdir(exist_ok=True)
#         self.classification_model = classification_model
#         self.embedding_model = embedding_model
#         self.label_binarizers = label_binarizers
#         self.metadata_path = self.cache_dir / 'generated_metadata.json'
#         self.category_stats_path = self.cache_dir / 'category_statistics.json'
#         self.metadata = {}
#         self.category_stats = {}
#         self.image_features = {}
        
#         self.generate_or_load_metadata()
        
#     def generate_or_load_metadata(self):
#         """Generate metadata for all images or load from cache"""
#         if self.metadata_path.exists():
#             print("Loading existing metadata...")
#             with open(self.metadata_path, 'r') as f:
#                 self.metadata = json.load(f)
#             with open(self.category_stats_p


ath, 'r') as f:
#                 self.category_stats = json.load(f)
#             print(f"Loaded metadata for {len(self.metadata)} images")
#             self._print_category_statistics()
#         else:
#             print("Generating new metadata for all images...")
#             self._generate_metadata()
    
#     def _generate_metadata(self):
#         """Process all images to generate metadata and category statistics"""
#         batch_size = 32
#         all_images = list(Path(self.base_image_dir).rglob('*.[jJ][pP][gG]'))
#         total_images = len(all_images)
        
#         self.category_stats = {
#             'total_images': total_images,
#             'categories': {},
#             'category_combinations': {}
#         }
        
#         print(f"Processing {total_images} images...")
        
#         for i in tqdm(range(0, total_images, batch_size), desc="Generating metadata"):
#             batch_paths = all_images[i:i + batch_size]
#             batch_arrays = []
#             valid_paths = []
            
#             for path in batch_paths:
#                 img_array = self.preprocess_image(path)
#                 if img_array is not None:
#                     batch_arrays.append(img_array[0])
#                     valid_paths.append(path)
            
#             if batch_arrays:
#                 batch_array = np.array(batch_arrays)
#                 classifications = self.classification_model.predict(batch_array)
#                 embeddings = self.embedding_model.predict(batch_array)
                
#                 for idx, path in enumerate(valid_paths):
#                     article_id = path.stem
                    
#                     categories = self._process_predictions(classifications[idx])
                    
#                     self.metadata[article_id] = {
#                         'path': str(path),
#                         'categories': categories['categories'],
#                         'confidence_scores': categories['scores'],
#                         'primary_category': categories['primary_category']
#                     }
                    
#                     self._update_category_stats(categories['categories'])
                    
#                     feature_path = self.cache_dir / f"{article_id}_features.npz"
#                     np.savez(
#                         feature_path,
#                         classifications=classifications[idx],
#                         embeddings=embeddings[idx]
#                     )
        
#         total = self.category_stats['total_images']
#         for cat, count in self.category_stats['categories'].items():
#             self.category_stats['categories'][cat] = {
#                 'count': count,
#                 'percentage': (count / total) * 100
#             }
        
#         with open(self.metadata_path, 'w') as f:
#             json.dump(self.metadata, f)
#         with open(self.category_stats_path, 'w') as f:
#             json.dump(self.category_stats, f)
        
#         self._print_category_statistics()
    
#     def _process_predictions(self, predictions):
#         """Process model predictions into categories with confidence scores"""
#         threshold = 0.3  
#         categories = []
#         scores = {}
        
#         category_names = ['mens', 'womens', 'children', 'shirts', 'pants', 'shoes', 'accessories']
#         for idx, score in enumerate(predictions):
#             if score > threshold:
#                 cat = category_names[idx]
#                 categories.append(cat)
#                 scores[cat] = float(score)
        
#         primary_category = max(scores.items(), key=lambda x: x[1])[0] if scores else None
        
#         return {
#             'categories': categories,
#             'scores': scores,
#             'primary_category': primary_category
#         }
    
#     def _update_category_stats(self, categories):
#         """Update category statistics"""
#         for cat in categories:
#             if cat not in self.category_stats['categories']:
#                 self.category_stats['categories'][cat] = 0
#             self.category_stats['categories'][cat] += 1
        
#         combo = '+'.join(sorted(categories))
#         if combo not in self.category_stats['category_combinations']:
#             self.category_stats['category_combinations'][combo] = 0
#         self.category_stats['category_combinations'][combo] += 1
    
#     def _print_category_statistics(self):
#         """Print category distribution statistics"""
#         print("\nCategory Distribution:")
#         for cat, stats in self.category_stats['categories'].items():
#             print(f"{cat}: {stats['count']} images ({stats['percentage']:.1f}%)")
        
#         print("\nTop Category Combinations:")
#         combinations = sorted(
#             self.category_stats['category_combinations'].items(),
#             key=lambda x: x[1],
#             reverse=True
#         )[:10]
#         for combo, count in combinations:
#             print(f"{combo}: {count} images")
    
#     def load_category_features(self, categories, limit=1000):
#         """Load features for specified categories with limit"""
#         relevant_ids = [
#             article_id for article_id, info in self.metadata.items()
#             if any(cat in info['categories'] for cat in categories)
#         ]
        
#         relevant_ids = sorted(
#             relevant_ids,
#             key=lambda x: max(self.metadata[x]['confidence_scores'].values()),
#             reverse=True
#         )[:limit]
        
#         print(f"Loading {len(relevant_ids)} features for categories: {categories}")
        
#         for article_id in tqdm(relevant_ids, desc="Loading features"):
#             feature_path = self.cache_dir / f"{article_id}_features.npz"
#             if feature_path.exists():
#                 with np.load(feature_path, allow_pickle=True) as data:
#                     self.image_features[article_id] = {
#                         'classifications': data['classifications'],
#                         'embeddings': data['embeddings']
#                     }

In [ ]:
# matcher = ClusteredImageMatcher(
#     classification_model=classification_model,
#     embedding_model=embedding_model,
#     label_binarizers=label_binarizers,
#     base_image_dir="images1"
# )

# recommendations = matcher.get_recommendations(
#     input_image_path="images/28865.jpg",
#     top_k=5
# )

# matcher.visualize_recommendations(
#     query_image_path="images/28865.jpg",
#     recommendations=recommendations,
#     title="Fashion Recommendations"
# )
# plt.show()